# AKSSF Anadromous Waters Catalog Data Join
Use spatial join to add AWC_CODE, Species and LSTAGE information to AKSSF catchments.  Add new field to concatenate species+lstage and field ot designate salmon spawning habitat presence or absence.

In [1]:
import os, arcpy, sys,datetime, traceback
import time

arcpy.env.overwriteOutput = True
sr = arcpy.SpatialReference(3338)  #'NAD_1983_Alaska_Albers'
arcpy.env.outputCoordinateSystem = sr

print('imports complete')
print(f'{("-"*100)}')
print(f'sys paths {sys.path}')
print(f'{("-"*100)}')
print(f'Python Environment set to - {sys.base_exec_prefix}')
print(f'{("-"*100)}')
print (datetime.datetime.now())
outdir = os.path.dirname(os.getcwd())
print(f'CSV table output directory set to {outdir}')

imports complete
----------------------------------------------------------------------------------------------------
sys paths ['C:\\Users\\dwmerrigan\\Documents\\GitHub\\AKSSF\\data_preparation\\sensitivity_drivers\\geomorphology', 'C:\\Program Files\\ArcGIS\\Pro\\Resources\\ArcPy', 'C:\\Users\\dwmerrigan\\Documents\\GitHub\\AKSSF', 'C:\\Program Files\\ArcGIS\\Pro\\bin\\Python\\envs\\arcgispro-py3\\python39.zip', 'C:\\Program Files\\ArcGIS\\Pro\\bin\\Python\\envs\\arcgispro-py3\\DLLs', 'C:\\Program Files\\ArcGIS\\Pro\\bin\\Python\\envs\\arcgispro-py3\\lib', 'C:\\Program Files\\ArcGIS\\Pro\\bin\\Python\\envs\\arcgispro-py3', '', 'C:\\Program Files\\ArcGIS\\Pro\\bin\\Python\\envs\\arcgispro-py3\\lib\\site-packages', 'C:\\Program Files\\ArcGIS\\Pro\\bin', 'C:\\Program Files\\ArcGIS\\Pro\\Resources\\ArcToolbox\\Scripts', 'C:\\Program Files\\ArcGIS\\Pro\\bin\\Python\\envs\\arcgispro-py3\\lib\\site-packages\\Babel-2.11.0-py3.9.egg', 'C:\\Program Files\\ArcGIS\\Pro\\bin\\Python\\envs\\arcgi

## Functions
Define functions that will be used throughout the script

In [2]:
# Function to add key, value pairs to dictionary
def append_value(dict_obj, key, value):
    # Check if key exist in dict or not
    if key in dict_obj:
        # Key exist in dict.
        # Check if type of value of key is list or not
        if not isinstance(dict_obj[key], list):
            # If type is not list then make it list
            dict_obj[key] = [dict_obj[key]]
        # Append the value in list
        dict_obj[key].append(value)
    else:
        # As key is not in dict,
        # so, add key-value pair
        dict_obj[key] = value
# Function to remove parenthesis from user inputs
def replace_all(userinput, dic):
    for i, j in dic.items():
        userinput = userinput.replace(i, j)
    return userinput

# Getnull rows from numpy array
def getnull(cat_ID_con):
    nullRows = []
    nullRows.append(cat_ID_con)
    return True

#Generate unique column names
def uniquify(df_final):
    seen = set()
    for item in df_final:
        fudge = 1
        newitem = item
        while newitem in seen:
            fudge += 1
            newitem = "{}_{}".format(item, fudge)
        yield newitem
        seen.add(newitem)

def move_comments_above_code(code):
    # Split the code into lines
    lines = code.split('\n')
    # Iterate over each line and look for comments at the end
    for i in range(len(lines)):
        if '#' in lines[i] and not lines[i].startswith('#'):
            comment_start = lines[i].find('#')
            # Insert the comment above the current line
            lines[i] = lines[i][:comment_start]  # Remove comment from current line
            comment = lines[i][comment_start:]
            lines[i] = lines[i][:comment_start]  # Remove comment from current line
            lines[i] = f"{lines[i].rstrip()}  {comment}\n"  # Add comment as a new line above current line
    # Join the modified lines back into a single string and return it
    return '\n'.join(lines)

def get_input(prompt, default):
    root = tk.Tk()
    root.withdraw()
    while True:
        input_text = simpledialog.askstring('Input', prompt, initialvalue=default)
        if input_text is None:
            # User clicked "Cancel"
            return None
        elif arcpy.Exists(input_text):
            # Path is valid, return it
            root.destroy()
            return input_text
        else:
            # Path is invalid, show error message and ask again
            error_message = f"{input_text} is not a valid path. Please try again."
            tk.messagebox.showerror("Error", error_message)



## Section 1 - Set input datasets and output locations
User to input paths for necessary input data and output locations
Update to add popup window


In [ ]:
import tkinter as tk
from tkinter import simpledialog

import arcpy
import sys
import os

# Define the default paths
default_paths = {
    'catsint': 'D:\\GIS\\outputs_2023\\outputs_2023.gdb\\AKSSF_awcHuc12_outlet_cats_merge',
    #'catsint': 'U:\\GIS\\AKSSF_awcHuc12_cv\\AKSSF_awcHuc12_cv.gdb\\AKSSF_awcHuc12_outlet_cats_merge',
    'awc_events_source': 'D:\\Basedata\\AWC\\AWC_2021_SpeciesEvents.gdb\\awcEventArcs',
    'awc_lakes': 'D:\\Basedata\\AWC\\AWC_2021_SpeciesEvents.gdb\\AWCLakeEvents',
    #'outputgdb': 'U:\\GIS\\AKSSF_awcHuc12_cv\\AKSSF_awcHuc12_cv.gdb',
    'outputgdb': 'D:\\GIS\\outputs_2023\\outputs_2023.gdb',
    #'sites': 'U:\\GIS\\AKSSF_awcHuc12_cv\\AKSSF_awcHuc12_cv.gdb\\AKSSF_awcHuc12_outlet_cats_points',
    'sites': 'D:\\GIS\\outputs_2023\\outputs_2023.gdb\\AKSSF_awcHuc12_outlet_cats_points',
}

# Replace inputDict with a single dictionary
inputDict = {}

# Get the user inputs
for key, value in default_paths.items():
    prompt = f'Input path to {key}.\nLeave blank and hit enter to use the default location.\nDefault = {value}\n'
    default = value
    default_paths[key] = get_input(prompt, default)
    append_value(inputDict,key,default_paths[key])

catsint = inputDict["catsint"]
awc_events_source = inputDict["awc_events_source"]
awc_lakes = inputDict["awc_lakes"]
outputgdb = inputDict["outputgdb"]
sites = inputDict["sites"]

print(f'AKSSF AWC HUC12 outlet catchments set to {catsint}\n {"-"*100}')
print(f'AWC events feature class set to {awc_events_source}\n {"-"*100}')
print(f'AWC lakes events feature class set to {awc_lakes}\n {"-"*100}')
print(f'Output GDB set to {outputgdb}\n {"-"*100}')
print(f'AKSSF AWC HUC12 catchment outlet centroids {sites}\n {"-"*100}')
print(f'Inputs set {datetime.datetime.now()}')


In [ ]:
# Old code
# # Get user inputs
# # Used to format user inputs
# inputDict = {"'":"",'"':""}# Specify path to AKSSF parent directory
#
# while True:
#     try:
#         userinput = replace_all((input('Input path to AWC HUC12 catchments outlets fc.\nLeave blank and hit enter to use the default location.\nDefault = U:\\GIS\\AKSSF_awcHuc12_cv\\AKSSF_awcHuc12_cv.gdb\\AKSSF_awcHuc12_outlet_cats_merge') or "U:\\GIS\\AKSSF_awcHuc12_cv\\AKSSF_awcHuc12_cv.gdb\\AKSSF_awcHuc12_outlet_cats_merge"),inputDict)
#         if not arcpy.Exists(userinput):
#             print('Path specified does not exist!\nPlease re-enter a valid path')
#             continue
#         else:
#             catsint = userinput
#             break
#     except KeyboardInterrupt:
#         print('interrupted!')
#         sys.exit()
# print(f'AKSSF AWC HUC12 outlet catchments set to {catsint}\n {"-"*100}')
#
# # Specify path to AWC events fc
# while True:
#     try:
#         userinput2 = replace_all((input('Input path to awc events feature class or shapefile.\nLeave blank and hit enter to use the default location.\nDefault = D:\\Basedata\\AWC\\AWC_2021_SpeciesEvents.gdb\\awcEventArcs') or "D:\\Basedata\\AWC\\AWC_2021_SpeciesEvents.gdb\\awcEventArcs"), inputDict)
#         if not arcpy.Exists(userinput2):
#             print('Path specified does not exist!\nPlease re-enter a valid path')
#             continue
#         else:
#             awc_events_source = userinput2
#             break
#     except KeyboardInterrupt:
#         print('interrupted!')
#         sys.exit()
# print(f'AWC events feature class set to {awc_events_source}\n {"-"*100}')
#
# # Specify path to AWC Lakes events fc
# while True:
#     try:
#         userinput4 = replace_all((input('Input path to awc lakes events feature class or shapefile.\nLeave blank and hit enter to use the default location.\nDefault = D:\\Basedata\\AWC\\AWC_2021_SpeciesEvents.gdb\\AWCLakeEvents') or "D:\\Basedata\\AWC\\AWC_2021_SpeciesEvents.gdb\\AWCLakeEvents"), inputDict)
#         if not arcpy.Exists(userinput4):
#             print('Path specified does not exist!\nPlease re-enter a valid path')
#             continue
#         else:
#             awc_lakes = userinput4
#             break
#     except KeyboardInterrupt:
#         print('interrupted!')
#         sys.exit()
# print(f'AWC lakes events feature class set to {awc_lakes}\n {"-"*100}')
#
# # Specify path to output gdb
# while True:
#     try:
#         userinput3 = replace_all((input('Input path to output gdb.\nLeave blank and hit enter to use the default location.\nDefault = U:\\GIS\\AKSSF_awcHuc12_cv\\AKSSF_awcHuc12_cv.gdb') or "U:\\GIS\\AKSSF_awcHuc12_cv\\AKSSF_awcHuc12_cv.gdb"), inputDict)
#         if not arcpy.Exists(userinput3):
#             print('Path specified does not exist!\nPlease re-enter a valid path')
#             continue
#         else:
#             outputgdb = userinput3
#             break
#     except KeyboardInterrupt:
#         print('interrupted!')
#         sys.exit()
# print(f'Output GDB set to {outputgdb}\n {"-"*100}')
#
# # Specify path to sites spatial join fc
# while True:
#     try:
#         siteinput = replace_all((input('Input path to temp sites fc.\nLeave blank and hit enter to use the default location.\nDefault = U:\\GIS\\AKSSF_awcHuc12_cv\\AKSSF_awcHuc12_cv.gdb\\AKSSF_awcHuc12_outlet_cats_points') or "U:\\GIS\\AKSSF_awcHuc12_cv\\AKSSF_awcHuc12_cv.gdb\\AKSSF_awcHuc12_outlet_cats_points"),inputDict)
#         if not arcpy.Exists(siteinput):
#             print('Path specified does not exist!\nPlease re-enter a valid path')
#             continue
#         else:
#             sites = siteinput
#             break
#     except KeyboardInterrupt:
#         print('interrupted!')
#         sys.exit()
# print(f'AKSSF AWC HUC12 catchment outlet centroids {sites}\n {"-"*100}')



In [ ]:
# Make copy of awcEvents in memory and add second OID field because original field will not honor the merge rule 'join' in arcpy spatial join (works in Pro?)
awc_events = arcpy.FeatureClassToFeatureClass_conversion(inputDict['awc_events_source'],r'memory','awcEventsCopy')
arcpy.AddField_management(awc_events,'OID2','LONG')
with arcpy.da.UpdateCursor(awc_events,['OBJECTID','OID2']) as cur:
    for row in cur:
        row[1] = row[0]
        cur.updateRow(row)
    del(row)
del(cur)
print(f'AWC Copy complete\n{"*"*100}')

In [ ]:
import time, datetime
arcpy.env.overwriteOutput = True
# Parameters
joinopt = "JOIN_ONE_TO_ONE"
jointype = "KEEP_ALL"
matchopt = "INTERSECT"
s_rad = "1 Centimeters"
processStart = time.time()

print(f'Joining species and lifestage data from {awc_events} to {catsint}')
awcjoin = arcpy.analysis.SpatialJoin(catsint,
                                     awc_events,
                                     os.path.join(outputgdb,'AKSSF_awcHuc12_cats_awc_spJoin'),
                                     joinopt,
                                     jointype,
                                     r'cat_ID_con "cat_ID_con" true true false 255 Text 0 0,First,#,catsint,cat_ID_con,0,255;'
                                     r'HUC12 "HUC12" true true false 255 Text 0 0,First,#,catsint,HUC12,0,255;'
                                     r'MERGE_SRC "MERGE_SRC" true true false 255 Text 0 0,First,#,catsint,MERGE_SRC,0,255;'
                                     # Add AWC_OID field so Sp/Ls records can be queried from polyine rather than AWC_CODE
                                     r'AWC_OID "AWC_OID" true true false 500 Text 0 0,Join,",",awcEventArcs,OID2,-1,-1;'
                                     r'AWC_CODE "AWC_CODE" true false false 500 Text 0 0,Join,",",awc_events,AWC_CODE,0,70;'
                                     r'SPECIES "SPECIES" true false false 500 Text 0 0,Join,",",awc_events,SPECIES,0,2;'
                                     r'LSTAGE "LSTAGE" true false false 500 Text 0 0,Join,",",awc_events,LSTAGE,0,1',
                                     matchopt,
                                     s_rad,
                                     ''
                                     )

# ADD additional fields
arcpy.AddField_management(awcjoin,'sp_ls','TEXT')
arcpy.AddField_management(awcjoin,'sa_sp_hab','SHORT')

arcpy.AddField_management(awcjoin,'sa_sp_count','SHORT')
arcpy.AddField_management(awcjoin,'mult_awc','SHORT')
arcpy.AddField_management(awcjoin,'AWC_CODE_qc','TEXT',"","",500)

# End timing
processEnd = time.time()
processElapsed = int(processEnd - processStart)
processSuccess_time = datetime.datetime.now()
print(f'Process complete')

# Report success
print(f'Process completed at {processSuccess_time.strftime("%Y-%m-%d %H:%M")} '
      f'(Elapsed time: {datetime.timedelta(seconds=processElapsed)})')
print(f'{"*"*100}')


## Make AWC relational dictionaries
Make 2 AWC events dictionaries. One with pk = AWC_CODE and second with pk = OBJECTID
Make AWC lakes dictionary with pk = AWC_CODE


In [ ]:
awcDict = {}
with arcpy.da.SearchCursor(awc_events, ['AWC_CODE',"SPECIES","LSTAGE"]) as cur:
    for row in cur:
        sp_ls_val  = row[1] + "_" + row[2]
        append_value(awcDict,row[0],sp_ls_val)

awcOidDict = {}
with arcpy.da.SearchCursor(awc_events, ['OBJECTID',"SPECIES","LSTAGE","AWC_CODE"]) as cur:
    for row in cur:
        sp_ls_val  = row[1] + "_" + row[2]
        append_value(awcOidDict,row[0],[sp_ls_val,row[3]])

awcLakesDict = {}
with arcpy.da.SearchCursor(awc_lakes, ['AWC_CODE',"SPECIES","LSTAGE"]) as cur:
    for row in cur:
        sp_ls_val  = row[1] + "_" + row[2]
        append_value(awcLakesDict,row[0],sp_ls_val)
print(f'AWC Dictionaries Created\n{("*"*100)}')


## Use update cursor to populate species fields
sa_sp_hab: 1 = spawning, 2 = salmon species present but not spawning, null = no awc records
sa_sp_count: # of salmon species listed as spawning, null for no data
mult_awc: 1 for multiple awc codes, 0 for one awc code and null for no data

In [ ]:
dropfields = ['Join_Count', 'NHDPlusID','catID','Shape','TARGET_FID','SOURCEFC','region']
keepfields = [f.name for f in arcpy.ListFields(awcjoin) if f.name not in dropfields]
sa_sp_codes = ['CO_s','S_s','K_s', 'CH_s','P_s']
sa_no_sp_codes = ['CO_p','CO_r','S_p','S_r','K_p','K_r','CH_p','CH_r','P_r','P_p']
print(keepfields)
multirecslist = []
multirecsDict = {}

norecslist = []
sp_lsList = []
awcCodeDict = {}
catAwcDict = {}
curfields = ['cat_ID_con','AWC_CODE','Species', 'LSTAGE', 'sp_ls','sa_sp_hab','mult_awc','AWC_CODE_qc','AWC_OID','sa_sp_count']
c2 = 0
processStart = time.time()
with arcpy.da.UpdateCursor(awcjoin, curfields) as cur:
    for row in cur:
        awcvals = []
        print(f'{(100*"*")}')
        if row[1] is not None:
            for val in row[1].split(','):
                val2 = "'"+str(val)+"'"
                awcvals.append(val)
            awcvals = list(set(awcvals))
            if len(str(awcvals).split(',')) > 1:
                print(f'Multiple awc records({len(str(awcvals).split(","))}) for this catchment ')
                append_value(multirecsDict,row[0],awcvals)
                row[6] = 1
                row[7] = str(awcvals).strip("[]").replace("'","")
                multirecslist .append(row[0])
            else:
                row[7]=awcvals[0]
                row[6] = 0
            spvals = row[2].split(",")
            lsvals = row[3].split(",")
            snum = len(row[2].split(","))
            cat_spList =[]
            for i in range(snum):
                sp_ls = spvals[i] + '_' + lsvals[i]
                append_value(awcCodeDict,row[1],sp_ls)
                append_value(catAwcDict,row[0],sp_ls)
                sp_lsList.append(sp_ls)
                cat_spList.append(sp_ls)
            print(f'{row[0]} has {snum} species_ls records {cat_spList}')
            print(f'list without duplicate records {list(set(cat_spList))}')
            row[4] = str(list(set(cat_spList))).strip("[]").replace("'","")
            if len(set(cat_spList).intersection(sa_sp_codes)) >0:
                print(f'{row[0]} has {set(cat_spList).intersection(sa_sp_codes)} salmon spawning records')
                saspcount = len(set(cat_spList).intersection(sa_sp_codes))
                row[5] = 1
                row[9] = saspcount
            elif len(set(cat_spList).intersection(sa_no_sp_codes)) > 0  and len(set(cat_spList).intersection(sa_sp_codes)) == 0:
                print(f'{row[0]} has {len(set(cat_spList).intersection(sa_no_sp_codes))} salmon species records')
                row[5] = 2
                row[9] = 0
            else:
                print(f'{row[0]} has no spawning habitat recorded')
                row[5] = 0
                row[9] = 0
        else:
            print(f'{row[0]} did not intersect the AWC ')
            row[1]=None
            row[2]=None
            row[3]=None
            row[4]=None
            row[5]=None
            row[6]=None
            row[7]=None
            row[8]=None
            row[9]=None
            norecslist.append(row[0])
            c2+=1
        cur.updateRow(row)
    del(row)
del(cur)

# End timing
processEnd = time.time()
processElapsed = int(processEnd - processStart)
processSuccess_time = datetime.datetime.now()
print(f'Process complete')

# Report success
print(f'Process completed at {processSuccess_time.strftime("%Y-%m-%d %H:%M")} '
      f'(Elapsed time: {datetime.timedelta(seconds=processElapsed)})')
print(f'{"*"*100}')

print(f'{c2} catchments did not have AWC species records')
print(norecslist)

# Qc and update results
Update using qc dictionaries from initial review, 2nd and final.
Rebuild watersheds before running script below and review again/create new watersheds as necessary.
Some outlet catchments appear to be wrong when compared to the flow accumulation grid but when compared to values stored in the vaa_merge (from which they are identified) you can see that the catchment identified is the one with the maximum TotDASqKM value...I think in most cases it makes sense to move the outlet catchment to the catchment that matches the fac grid/imagery unless the new watershed from any change will not be significantly different from the original.

Example of the vaa vs flow acc grid discrepancy (calculated values from fac count much different than vaa totdasqkm value):
 * TotDASqKm of Cook_Inlet_75005300002575 = 2028.669626 but fac value @ 1,142,521 (1142521 x 25sqm x 1e-6 = 28.56 sqkm) # outlet catchment identified from vaa table values

 * TotDASqKm of Cook_Inlet_75005300009967 = 2000.180576 but fac value @ 21,512,896 (21512896 x 25sqm x 1e-6 = 537.82 sqkm) # catchment that should be outlet based on visual examination and flow acc grid values

 * Check of Cook_Inlet_75005300016711 = 41.32sqkm , fac @ 1,405,319 (1405319 x 25sqm x 1e-6 = 35.13)
 * Check of Cook_Inlet_75004400008887 = 11.45sqkm, fac @ 458,122 (458122 x 25sqm x 1e-6 = 11.45)

Check Results
Check Multiple codes catchment - Cook_Inlet_75005300002504 "Straight Creek" had 2 AWC-Codes associated with the catchment during initial Spatial Join and should have the following records after qc update:
from manual Count : K_r, CH_p, S_r, CO_p, DV_r, P_p, DV_s
from QC script : K_r, CH_p, S_r, CO_p, DV_r, P_p, DV_s

Check missed during spatial join - 'Copper_River_75003900054448' AWC record begins above the outlet catchment:
Manual Count : P_p, CO_r, CT_s, CT_r, CO_s
QC Script result: P_p, CO_r, CT_s, CT_r, CO_s

Check new Watershed - 'Prince_William_Sound_77992':['Prince_William_Sound_75842'] where 75842 will be the new watershed/outlet catchment for the HUC that initially had 77992 as the outlet.
Manual Count : P_s, Co_s, CO_r
QC Script result: P_s, CO_s, CO_r

Check new Watershed that did not intersect AWC - 'Copper_River_75003900054944':['Copper_River_75003900054448']
Manual Count : P_p, CO_r, CT_s, CT_r, CO_s
QC Script result: P_p, CO_r, CT_s, CT_r, CO_s


### Compare dictionaries and use to set definition queries for review.


I think this block is from the orignal 2nd round of QC - revisit when ready for second round qc on Kodiak and BBay

In [ ]:
# alreadyQCList = []
# for k, v in awcHuc12QcDict.items():
#     print (f'"{k}",')
#     alreadyQCList.append(k)
# print(len(norecslist))
# print(len(multirecslist))
# qcList = norecslist + multirecslist
# print (len(qcList))
# print (f'qcList: {len(qcList)}') # List of all watersheds/catchments that need to be reviewed for either wrong catchment or multi_awc records.
# upQcList = [x for x in qcList if x not in alreadyQCList]
# print(len(upQcList))
# print(upQcList)
#
# # Drops from initial review.  Add these to review dictionary below
# dropList = ['Cook_Inlet_75004300001512', 'Cook_Inlet_75004300000779', 'Cook_Inlet_75004400008539', 'Cook_Inlet_75004400009109', 'Cook_Inlet_75004400002718', 'Cook_Inlet_75000100002639', 'Cook_Inlet_75000100004624', 'Cook_Inlet_75000700037019', 'Cook_Inlet_75000500015568', 'Cook_Inlet_75000500006160', 'Cook_Inlet_75000500009986', 'Cook_Inlet_75000300007777', 'Cook_Inlet_75000600014946', 'Cook_Inlet_75000600020425', 'Cook_Inlet_75000200015292', 'Cook_Inlet_75005300000657', 'Cook_Inlet_75005400045669', 'Cook_Inlet_75005400028381', 'Cook_Inlet_75005400023457', 'Cook_Inlet_75005400047280', 'Copper_River_75019800002596', 'Copper_River_75019800019137', 'Copper_River_75019800020572', 'Copper_River_75019800000761', 'Copper_River_75019600178680', 'Copper_River_75019600201122', 'Copper_River_75019600145942', 'Copper_River_75019600119614', 'Copper_River_75019600130736', 'Copper_River_75019600198047', 'Copper_River_75019700000617', 'Copper_River_75019700000923', 'Copper_River_75019700003395', 'Copper_River_75003900073776', 'Copper_River_75003900066004', 'Copper_River_75003900054248', 'Copper_River_75003900029086', 'Prince_William_Sound_33616', 'Prince_William_Sound_91751']
#
# upQcList2 = [x for x in upQcList if x not in dropList]
# #print(upQcList2)
# #format for key and print to add
# for item in dropList:
#     print(f"'{item}':None,")


## Review dictionaries
Create Dictionaries to store missed hucs, catchments without awc relations, wrong watersheds, and Hucs that should be excluded

### CookInlet, Copper River,and PWS
Initial Review dictionary

In [ ]:
# Dictionary of all catchments to be reviewed either for Multiple code associations, new catchments/watersheds or just missed during spatial join

# awcHuc12QcDict = {'Cook_Inlet_75004400002041': '244-30-10010-2177-3012',
#                   'Cook_Inlet_75000200007660': '247-41-10100',
#                   'Cook_Inlet_75004400011322': '247-60-10170-2029-3033',
#                   'Cook_Inlet_75004300006013': '244-30-10050-2059',
#                   'Cook_Inlet_75004400010320': '244-30-10010-2030',
#                   'Copper_River_75019800010313': '212-20-10080-2655',
#                   'Cook_Inlet_75004400010754': '244-30-10010-2158',
#                   'Copper_River_75019700004190': '212-20-10080-2300-3421',
#                   'Cook_Inlet_75004400010402': '244-30-10010-2025-3041',
#                   'Cook_Inlet_75004300001162': '244-10-10010-2011',
#                   'Cook_Inlet_75000100002463': '247-50-10060',
#                   'Prince_William_Sound_43933': '221-20-10360',
#                   'Cook_Inlet_75004200001493': '233-20-10080',
#                   'Cook_Inlet_75004200000064': '231-30-10080-2010',
#                   'Cook_Inlet_75004300004365': '244-20-10100-2019',
#                   'Cook_Inlet_75004300003713': '244-20-10100',
#                   'Cook_Inlet_75004300002755': '244-20-10100-2045',
#                   'Cook_Inlet_75004300002023': '241-14-10645',
#                   'Cook_Inlet_75004300002027': '241-14-10645-2060',
#                   'Cook_Inlet_75004300001228': '244-10-10010-2011-3031',
#                   'Cook_Inlet_75004400011230': '244-30-10010-2250-3030',
#                   'Cook_Inlet_75004400002953': '247-90-10020-2058',
#                   'Cook_Inlet_75004400007512': '244-30-10010-2063',
#                   'Cook_Inlet_75004400009198': '247-60-10110-2120',
#                   'Cook_Inlet_75004400011676': '247-60-10220',
#                   'Cook_Inlet_75004400004680': '244-30-10010-2076',
#                   'Cook_Inlet_75004400001464': '247-60-10170-2029-3041',
#                   'Cook_Inlet_75004400001136': '247-60-10250',
#                   'Cook_Inlet_75004400010328': '244-30-10010',
#                   'Cook_Inlet_75004400004743': '247-60-10100',
#                   'Cook_Inlet_75004400010352': '247-60-10150-2025',
#                   'Cook_Inlet_75004400010337': '244-30-10010-2063-3041',
#                   'Cook_Inlet_75004400010221': '247-60-10230',
#                   'Cook_Inlet_75004400009412': '244-30-10010-2177',
#                   'Cook_Inlet_75004400005344': '244-30-10010-2225-3031',
#                   'Cook_Inlet_75004400008192': '247-60-10170-2029',
#                   'Cook_Inlet_75004400002258': '247-60-10280',
#                   'Cook_Inlet_75004400008859': '247-90-10020',
#                   'Cook_Inlet_75004400000153': '247-90-10020',
#                   'Cook_Inlet_75004400004707': '247-60-10110-2080',
#                   'Cook_Inlet_75004400007493': '244-30-10010-2050',
#                   'Cook_Inlet_75004400002957': '244-30-10010-2063-3036',
#                   'Cook_Inlet_75000100001880': '247-60-10340-2018',
#                   'Cook_Inlet_75000100002688': '247-50-10150',
#                   'Cook_Inlet_75000100004381': '247-60-10320',
#                   'Cook_Inlet_75000100002741': '247-50-10110-2070',
#                   'Cook_Inlet_75000100000953': '247-50-10175-2025',
#                   'Cook_Inlet_75000400034535': '247-50-10200-2160',
#                   'Cook_Inlet_75000400001221': '247-50-10220',
#                   'Cook_Inlet_75000700033509': '247-41-10200-2391',
#                   'Cook_Inlet_75000700049477': '247-41-10200-2551',
#                   'Cook_Inlet_75000700036340': '247-41-10200-2596',
#                   'Cook_Inlet_75000700044263': '247-41-10200',
#                   'Cook_Inlet_75000700049829': '247-41-10200-2696-3020',
#                   'Cook_Inlet_75000700036399': '247-41-10200-2585-3223',
#                   'Cook_Inlet_75000500008473': '247-41-10200-2381-3161-4016',
#                   'Cook_Inlet_75000500009358': '247-41-10200-2381-3235',
#                   'Cook_Inlet_75000500005048': '247-41-10200-2381-3231',
#                   'Cook_Inlet_75000500013555': '247-41-10200-2381-3240',
#                   'Cook_Inlet_75000500008026': '247-41-10200-2381',
#                   'Cook_Inlet_75000300001605': '247-41-10200-2370-3301-4044',
#                   'Cook_Inlet_75000300007234': '247-41-10200-2370-3015',
#                   'Cook_Inlet_75000300018625': '247-41-10200-2370',
#                   'Cook_Inlet_75000300025330': '247-41-10200-2370',
#                   'Cook_Inlet_75000600003641': '247-41-10200-2053-3250',
#                   'Cook_Inlet_75000600035022': '247-41-10200-2053-3213',
#                   'Cook_Inlet_75000600024655': '247-41-10200-2053-3205-4053-5066',
#                   'Cook_Inlet_75000600034967': '247-41-10200-2053-3205-4053-5066',
#                   'Cook_Inlet_75000600040180': '247-41-10200-2053-3205-4053-5072',
#                   'Cook_Inlet_75000600025547': '247-41-10200-2053',
#                   'Cook_Inlet_75000600004193': '247-41-10200-2053-3205-4053-5046-6011',
#                   'Cook_Inlet_75000600009435': '247-41-10200-2053-3205-4053-5046',
#                   'Cook_Inlet_75000600014556': '247-41-10200-2053-3220-4030-5040',
#                   'Cook_Inlet_75000600025489': '247-41-10200-2053-3229-4050',
#                   'Cook_Inlet_75000600034555': '247-41-10200-2053-3238',
#                   'Cook_Inlet_75000600023013': '247-41-10200-2053-3229',
#                   'Cook_Inlet_75000600034913': '247-41-10200-2053-3205-4053-5028',
#                   'Cook_Inlet_75000600015366': '247-41-10200-2053-3205',
#                   'Cook_Inlet_75000600022522': '247-41-10200-2053-3205-4112',
#                   'Cook_Inlet_75000600040654': '247-41-10200-2053-3150-4018',
#                   'Cook_Inlet_75000600014918': '247-41-10200-2053-3205-4009',
#                   'Cook_Inlet_75000600014428': '247-41-10200-2053-3205-4064',
#                   'Cook_Inlet_75000600036866': '247-41-10200-2053-3205-4112-5037',
#                   'Cook_Inlet_75000600012683': '247-41-10200-2053-3205',
#                   'Cook_Inlet_75000600005715': '247-41-10200-2053-3205-4112-5060',
#                   'Cook_Inlet_75000600020443': '247-41-10200-2053-3170-4057',
#                   'Cook_Inlet_75000600020381': '247-41-10200-2053-3150-4060-5033',
#                   'Cook_Inlet_75000200008600': '247-41-10200-2200-3050',
#                   'Cook_Inlet_75000200018499': '247-41-10200-2291-3011',
#                   'Cook_Inlet_75000200015565': '247-41-10200-2081-3065-4021',
#                   'Cook_Inlet_75000200005720': '247-41-10200-2081-3050-4040',
#                   'Cook_Inlet_75000200001253': '247-41-10200-2120',
#                   'Cook_Inlet_75000200001139': '247-41-10200-2250-3061-4009',
#                   'Cook_Inlet_75000200008048': '247-41-10200-2081-3100-4155',
#                   'Cook_Inlet_75000200015589': '247-41-10200-2020',
#                   'Cook_Inlet_75000200015485': '247-41-10200-2015-3025',
#                   'Cook_Inlet_75000200015301': '247-41-10200-2015-3035',
#                   'Cook_Inlet_75000200005066': '247-41-10200-2015-3040',
#                   'Cook_Inlet_75000200008267': '247-41-10200',
#                   'Cook_Inlet_75000200013093': '247-41-10200-2053-3020-4015',
#                   'Cook_Inlet_75005300026455': '247-20-10010-2040',
#                   'Cook_Inlet_75005300002504': '247-10-10080-2010-3040',
#                   'Cook_Inlet_75005300030509': '247-30-10090-2105-3015',
#                   'Cook_Inlet_75005300019114': '247-30-10080',
#                   'Cook_Inlet_75005300026951': '247-10-10080',
#                   'Cook_Inlet_75005300026568': '247-20-10010-2020',
#                   'Cook_Inlet_75005300030350': '247-20-10010-2052',
#                   'Cook_Inlet_75005300021072': '245-50-10010-2047',
#                   'Cook_Inlet_75005300003835': '247-20-10010',
#                   'Cook_Inlet_75005300022240': '247-10-10200',
#                   'Cook_Inlet_75005300024421': '245-50-10050',
#                   'Cook_Inlet_75005400015980': '245-10-10030-2007',
#                   'Cook_Inlet_75005400041329': '245-40-10020',
#                   'Copper_River_75019800005498': '212-20-10080-2481-3181',
#                   'Copper_River_75019800011243': '212-20-10080-2571-3062-4042',
#                   'Copper_River_75019800010136': '212-20-10080-2605-3019',
#                   'Copper_River_75019800006994': '212-20-10080-2571-3062',
#                   'Copper_River_75019800009388': '212-20-10080-2605',
#                   'Copper_River_75019800020803': '212-20-10080-2576',
#                   'Copper_River_75019800000492': '212-20-10080-2481-3071',
#                   'Copper_River_75019800003840': '212-20-10080-2481',
#                   'Copper_River_75019800011423': '212-20-10080-2755',
#                   'Copper_River_75019800011406': '212-20-10080-2500',
#                   'Copper_River_75019800011978': '212-20-10080-2481',
#                   'Copper_River_75019600164515': '212-20-10080-2461-3171',
#                   'Copper_River_75019600143415': '212-20-10080-2461-3091',
#                   'Copper_River_75019600187749': '212-20-10080-2461-3091-4122',
#                   'Copper_River_75019600201378': '212-20-10080-2461-3091-4042',
#                   'Copper_River_75019600117650': '212-20-10080-2461-3091',
#                   'Copper_River_75019600187874': '212-20-10080-2461-3091',
#                   'Copper_River_75019600165863': '212-20-10080-2461-3091-4021',
#                   'Copper_River_75019600118256': '212-20-10080-2461',
#                   'Copper_River_75019600113018': '212-20-10080-2461-3023',
#                   'Copper_River_75019600198311': '212-20-10080-2431-3051',
#                   'Copper_River_75019600117903': '212-20-10080',
#                   'Copper_River_75019600128647': '212-20-10080',
#                   'Copper_River_75019600195594': '212-20-10080-2331-3060',
#                   'Copper_River_75019600132577': '212-20-10080-2331-3068',
#                   'Copper_River_75019600119855': '212-20-10080-2331-3071',
#                   'Copper_River_75019600133541': '212-20-10080-2331',
#                   'Copper_River_75019600112990': '212-20-10080-2331-3081-4050',
#                   'Copper_River_75019700004212': '212-20-10080-2300-3540-4100',
#                   'Copper_River_75019700001727': '212-20-10080-2300-3310-4085',
#                   'Copper_River_75019700004866': '212-20-10080-2300-3540',
#                   'Copper_River_75019700017989': '212-20-10080-2300',
#                   'Copper_River_75019700005942': '212-20-10080-2300-3421-4021',
#                   'Copper_River_75019700010631': '212-20-10080-2300-3511',
#                   'Copper_River_75019700003637': '212-20-10080-2300-3371-4041',
#                   'Copper_River_75003900057793': '212-20-10080',
#                   'Copper_River_75003900034254': '212-20-10080-2159',
#                   'Copper_River_75003900051444': '212-20-10040',
#                   'Copper_River_75003900023132': '212-10-10010-2061',
#                   'Prince_William_Sound_18547': '227-10-17060-2012',
#                   'Prince_William_Sound_18737': '227-30-17790',
#                   'Prince_William_Sound_30886': '227-20-17386',
#                   'Prince_William_Sound_41995': '227-20-17590-2003',
#                   'Prince_William_Sound_43195': '228-60-18120',
#                   'Prince_William_Sound_45835': '224-40-14870',
#                   'Prince_William_Sound_74542': '222-50-12570',
#                   'Prince_William_Sound_79792': '223-30-13220',
#                   'Prince_William_Sound_91741': '221-60-11370',
#                   'Prince_William_Sound_93261': '221-60-11470',
#                   'Copper_River_75019700004288': None,
#                   'Cook_Inlet_75004400008887': None,
#                   'Cook_Inlet_75000100002639': None,
#                   'Cook_Inlet_75004300001512': None,
#                   'Cook_Inlet_75004300000779': None,
#                   'Cook_Inlet_75004400008539': None,
#                   'Cook_Inlet_75004400009109': None,
#                   'Cook_Inlet_75004400002718': None,
#                   'Cook_Inlet_75000100004624': None,
#                   'Cook_Inlet_75000700037019': None,
#                   'Cook_Inlet_75000500015568': None,
#                   'Cook_Inlet_75000500006160': None,
#                   'Cook_Inlet_75000500009986': None,
#                   'Cook_Inlet_75000300007777': None,
#                   'Cook_Inlet_75000600014946': None,
#                   'Cook_Inlet_75000600020425': None,
#                   'Cook_Inlet_75000200015292': None,
#                   'Cook_Inlet_75005300000657': None,
#                   'Cook_Inlet_75005400045669': None,
#                   'Cook_Inlet_75005400028381': None,
#                   'Cook_Inlet_75005400023457': None,
#                   'Cook_Inlet_75005400047280': None,
#                   'Copper_River_75019800002596': None,
#                   'Copper_River_75019800019137': None,
#                   'Copper_River_75019800020572': None,
#                   'Copper_River_75019800000761': None,
#                   'Copper_River_75019600178680': None,
#                   'Copper_River_75019600201122': None,
#                   'Copper_River_75019600145942': None,
#                   'Copper_River_75019600119614': None,
#                   'Copper_River_75019600130736': None,
#                   'Copper_River_75019600198047': None,
#                   'Copper_River_75019700000617': None,
#                   'Copper_River_75019700000923': None,
#                   'Copper_River_75019700003395': None,
#                   'Copper_River_75003900073776': None,
#                   'Copper_River_75003900066004': None,
#                   'Copper_River_75003900054248': None,
#                   'Copper_River_75003900029086': None,
#                   'Prince_William_Sound_33616': None,
#                   'Prince_William_Sound_91751': None,
#                   'Cook_Inlet_75000200005366': None,
#                   'Copper_River_75003900043422': None,
#                   'Copper_River_75003900033530': None,
#                   'Cook_Inlet_75004300002143': None,
#                   'Cook_Inlet_75000100000043': None,
#                   'Cook_Inlet_75000100002150': None,
#                   'Copper_River_75003900029096': None,
#                   'Prince_William_Sound_46513': None,
#                   'Cook_Inlet_75004200003619': '232-22-10130',
#                   'Cook_Inlet_75004200016346': '231-20-10337',
#                   'Cook_Inlet_75004200012680': '233-30-10070',
#                   'Cook_Inlet_75004200005429': '231-30-10080-2119',
#                   'Cook_Inlet_75004200012771': '242-31-10120',
#                   'Cook_Inlet_75004200006975': '226-50-16286',
#                   'Cook_Inlet_75004300006970': '241-14-10645',
#                   'Cook_Inlet_75004300002057': '241-14-10645-2131',
#                   'Cook_Inlet_75004300007978': '241-14-10600',
#                   'Cook_Inlet_75004300001135': '241-30-10500',
#                   'Cook_Inlet_75004300006302': '241-20-10550',
#                   'Cook_Inlet_75004300002183': '241-40-10300',
#                   'Cook_Inlet_75004300005952': '241-14-10660',
#                   'Cook_Inlet_75004400007224': '247-60-10210',
#                   'Cook_Inlet_75004400000334': '244-30-10010',
#                   'Cook_Inlet_75004400000370': '244-30-10010-2140',
#                   'Cook_Inlet_75004400003230': '244-30-10010-2076-3095',
#                   'Cook_Inlet_75004400001670': '244-30-10010-2096',
#                   'Cook_Inlet_75000100004048': '247-50-10175',
#                   'Cook_Inlet_75000100002482': '247-41-10080',
#                   'Cook_Inlet_75000100004383': '247-50-10330',
#                   'Cook_Inlet_75000400014408': '247-50-10200',
#                   'Cook_Inlet_75000400000017': '247-50-10220',
#                   'Cook_Inlet_75000400000163': '247-50-10220-2095',
#                   'Cook_Inlet_75000700031304': '247-41-10200-2713',
#                   'Cook_Inlet_75000700046720': '247-41-10200',
#                   'Cook_Inlet_75000500006217': '247-41-10200-2381-3161',
#                   'Cook_Inlet_75000500002477': '247-41-10200-2381-3161',
#                   'Cook_Inlet_75000600029695': '247-41-10200-2053-3170-4093',
#                   'Cook_Inlet_75000600027457': '247-41-10200-2053-3229',
#                   'Cook_Inlet_75000600002850': '247-41-10200-2053-3205-4075-5255',
#                   'Cook_Inlet_75000600020073': '247-41-10200-2053-3205',
#                   'Cook_Inlet_75000600027807': '247-41-10200-2053-3249',
#                   'Cook_Inlet_75000600020024': '247-41-10200-2053-3150',
#                   'Cook_Inlet_75000600039012': '247-41-10200-2053-3250',
#                   'Cook_Inlet_75000200008668': '247-41-10200-2250-3020',
#                   'Cook_Inlet_75000200010216': '247-41-10200-2015-3035-4223',
#                   'Cook_Inlet_75000200010370': '247-41-10200',
#                   'Cook_Inlet_75000200018431': '247-41-10100-2005',
#                   'Cook_Inlet_75005300016711': '245-50-10120',
#                   'Cook_Inlet_75005300002575': '245-50-10010',
#                   'Cook_Inlet_75005300019947': '247-10-10080-2010-3068',
#                   'Cook_Inlet_75005300002502': '247-10-10080-2010',
#                   'Cook_Inlet_75005300009788': '247-10-10080-2020-3029',
#                   'Cook_Inlet_75005300010442': '247-10-10080',
#                   'Cook_Inlet_75005300005137': '245-50-10050-2016-3044',
#                   'Cook_Inlet_75005300029149': '245-50-10020',
#                   'Cook_Inlet_75005300003369': '247-30-10090',
#                   'Cook_Inlet_75005300027433': '247-10-10080-2010-3060',
#                   'Cook_Inlet_75005400035306': '245-30-10010',
#                   'Cook_Inlet_75005400033392': '248-20-10070',
#                   'Cook_Inlet_75005400040621': '245-30-10090',
#                   'Cook_Inlet_75005400017221': '245-30-10120',
#                   'Cook_Inlet_75005400045707': '248-30-10010',
#                   'Cook_Inlet_75005400010388': '248-30-10020',
#                   'Cook_Inlet_75005400012302': '243-10-10030',
#                   'Cook_Inlet_75005400032742': '248-40-10100-2040',
#                   'Cook_Inlet_75005400039143': '248-40-10100',
#                   'Cook_Inlet_75005400038974': '248-20-10040',
#                   'Cook_Inlet_75005400031128': '243-10-10050',
#                   'Cook_Inlet_75005400029404': '245-40-10050',
#                   'Copper_River_75019800015373': '212-20-10080',
#                   'Copper_River_75019800001873': '212-20-10080',
#                   'Copper_River_75019800007832': '212-20-10080',
#                   'Copper_River_75019600181061': '212-20-10080-2461-3292',
#                   'Copper_River_75019600161870': '212-20-10080-2404',
#                   'Copper_River_75019600128599': '212-20-10080',
#                   'Copper_River_75019600197497': '212-20-10080-2331-3093',
#                   'Copper_River_75019700016379': '212-20-10080-2300-3660-4258',
#                   'Copper_River_75019700016412': '212-20-10080-2300-3660-4258-5050',
#                   'Copper_River_75019700010633': '212-20-10080-2300-3511-4035',
#                   'Copper_River_75019700010617': '212-20-10080-2300',
#                   'Copper_River_75019700007155': '212-20-10080-2300-3511-4110',
#                   'Copper_River_75019700006876': '212-20-10080-2300',
#                   'Copper_River_75019700012433': '212-20-10080-2300-3310',
#                   'Copper_River_75019700010666': '212-20-10080-2300-3511',
#                   'Copper_River_75003900065861': '212-20-10080-2055',
#                   'Copper_River_75003900049180': '212-20-10040-2011',
#                   'Copper_River_75003900055856': '212-20-10040',
#                   'Copper_River_75003900084262': '212-20-10080',
#                   'Prince_William_Sound_18357': '227-10-17110',
#                   'Prince_William_Sound_30774': '225-20-15030',
#                   'Prince_William_Sound_41123': '221-30-10560',
#                   'Prince_William_Sound_41143': '221-30-10655',
#                   'Prince_William_Sound_43513': '221-30-10517',
#                   'Prince_William_Sound_43793': '221-30-10480',
#                   'Prince_William_Sound_44113': '221-20-10350',
#                   'Prince_William_Sound_45863': '221-20-10260',
#                   'Prince_William_Sound_46065': '228-50-18190',
#                   'Prince_William_Sound_62232': '222-20-12242',
#                   'Prince_William_Sound_66912': '221-50-11060',
#                   'Prince_William_Sound_72972': '221-40-10990',
#                   'Prince_William_Sound_82471': '221-60-11380-2095',
#                   'Copper_River_75003900034359': '212-10-10020',
#                   'Cook_Inlet_75000600015299':'247-41-10200-2053-3150',
#                   'Cook_Inlet_75005300022592':'247-30-10090-2151',
#                   'Cook_Inlet_75005400039351':'248-40-10105',
#                   'Copper_River_75003900084672':'212-20-10080-2100-3089',
#                   'Copper_River_75003900054448':'228-70-10500',
#                   'Prince_William_Sound_39255':'224-40-14904',
#                   'Prince_William_Sound_60732':'223-30-13180',
#                   'Prince_William_Sound_78912':'222-10-12040'
#                   }
norecslist


### QC Dictionaries Created During Review

In [ ]:
# addHuc12QcDict = {'Cook_Inlet_75004200003619':['20661','20746','21215','22051'],
#                   'Prince_William_Sound_78912':['29546','30080','30343','30608'],
#                   'Prince_William_Sound_39255':['28534','28669'],
#                   'Prince_William_Sound_60732':['28070'],
#                   'Copper_River_75003900054448':['29073','29201','29280','30050','30133'],
#                   'Copper_River_75003900084672':['30582'],
#                   'Cook_Inlet_75005300022592':['26431'],
#                   'Cook_Inlet_75005400039351':['12237','12953'],
#                   'Cook_Inlet_75000600015299':['30863','31442','31536','31760','32110','32597'],
#                   'Cook_Inlet_75004200016346':['21668'],
#                   'Cook_Inlet_75004200012680':['22252','22404'],
#                   'Cook_Inlet_75004200005429':['20580'],
#                   'Cook_Inlet_75004200012771':['20608','20671','20894','21542','21682','22151'],
#                   'Cook_Inlet_75004200006975':['20808','20946','21328','21462','21569','21662','21736','22023','22198'],
#                   'Cook_Inlet_75004300006970':['20471','20625','20711','20913','20938','21141','21608','22191','22391'],
#                   'Cook_Inlet_75004300002057':['21250','21295','22039'],
#                   'Cook_Inlet_75004300007978':['18653','19249','19635','20198'],
#                   'Cook_Inlet_75004300001135':['18194','19433','19603','19984','20340'],
#                   'Cook_Inlet_75004300006302':['20654','20739','20811','20878','22135'],
#                   'Cook_Inlet_75004300002183':['12185','12958'],
#                   'Cook_Inlet_75004300005952':['20482','21102','21152','22394'],
#                   'Cook_Inlet_75004400007224':['27445','27587','27931','28237','28656'],
#                   'Cook_Inlet_75004400000334':['24422','24714','24964','25460','25481','25563','25756'],
#                   'Cook_Inlet_75004400000370':['27720','28378'],
#                   'Cook_Inlet_75004400003230':['20542','21948','22172'],
#                   'Cook_Inlet_75004400001670':['28363'],
#                   'Cook_Inlet_75000100004048':['33457','33912','34455','34669','34858','34962','35155'],
#                   'Cook_Inlet_75000100002482':['26579'],
#                   'Cook_Inlet_75000100004383':['32996','33227','33575','33632','34172','34973'],
#                   'Cook_Inlet_75000400014408':['33736','33902','34679','34740','35165'],
#                   'Cook_Inlet_75000400000017':['33068','33279','33659','34330','34494'],
#                   'Cook_Inlet_75000400000163':['33316','33882','34767'],
#                   'Cook_Inlet_75000700031304':['36420'],
#                   'Cook_Inlet_75000700046720':['27009','27197','27668','27827','28158','28225'],
#                   'Cook_Inlet_75000500006217':['35350','36473','36606','36695','37240'],
#                   'Cook_Inlet_75000500002477':['35350','36282','36473','36606','36695','37240'],
#                   'Cook_Inlet_75000600029695':['30958','31901'],
#                   'Cook_Inlet_75000600027457':['30722','31822','32118','32763','32952'],
#                   'Cook_Inlet_75000600002850':['32435'],
#                   'Cook_Inlet_75000600020073':['31065','31964','32006','32456','32809'],
#                   'Cook_Inlet_75000600027807':['31855'],
#                   'Cook_Inlet_75000600020024':['32022','32585'],
#                   'Cook_Inlet_75000600039012':['31641','31747'],
#                   'Cook_Inlet_75000200008668':['33219','33894','34732','34840'],
#                   'Cook_Inlet_75000200010216':['31946','32775'],
#                   'Cook_Inlet_75000200010370':['27009','27197','27434','2750','27668','27827','28225','28556','28645'],
#                   'Cook_Inlet_75000200018431':['26885','28430'],
#                   'Cook_Inlet_75005300016711':['24551','25423','26283'],
#                   'Cook_Inlet_75005300002575':['24545','24571','25723','26187','26741'],
#                   'Cook_Inlet_75005300019947':['24574','25394','25727'],
#                   'Cook_Inlet_75005300002502':['24443','24737','24907','25091','25124','25203','25586','26055','26457'],
#                   'Cook_Inlet_75005300009788':['24468','24874','25895','26002','26081','26359','26573'],
#                   'Cook_Inlet_75005300010442':['24509','24892','24971','24998','25186','25682','25762','25854','26039'],
#                   'Cook_Inlet_75005300005137':['25985'],
#                   'Cook_Inlet_75005300029149':['25438','26091'],
#                   'Cook_Inlet_75005300003369':['24526','24818','25338','25781','25950','25981','26388'],
#                   'Cook_Inlet_75005300027433':['24991','25239','25347'],
#                   'Cook_Inlet_75005400035306':['18227','18608','18707','19062','20120','20232'],
#                   'Cook_Inlet_75005400033392':['19023','19245','19323'],
#                   'Cook_Inlet_75005400040621':['18898'],
#                   'Cook_Inlet_75005400017221':['18334'],
#                   'Cook_Inlet_75005400045707':['18169','18628','19005'],
#                   'Cook_Inlet_75005400010388':['18275','18736','19763'],
#                   'Cook_Inlet_75005400012302':['11529','12346','12428','12458','12887','13146'],
#                   'Cook_Inlet_75005400032742':['11766','12060'],
#                   'Cook_Inlet_75005400039143':['11859','12352'],
#                   'Cook_Inlet_75005400038974':['19301','19388','20130','20339'],
#                   'Cook_Inlet_75005400031128':['10743','11237','11559','11938'],
#                   'Cook_Inlet_75005400029404':['18814','19385','19631','19664','20090'],
#                   'Copper_River_75019800015373':['29614','29821'],
#                   'Copper_River_75019800001873':['29821'],
#                   'Copper_River_75019800007832':['29614','29821'],
#                   'Copper_River_75019600181061':['36233'],
#                   'Copper_River_75019600161870':['33024'],
#                   'Copper_River_75019600128599':['28901','29408','29475','29614','29821','30538'],
#                   'Copper_River_75019600197497':['34150'],
#                   'Copper_River_75019700016379':['29090'],
#                   'Copper_River_75019700016412':['29009'],
#                   'Copper_River_75019700010633':['33929'],
#                   'Copper_River_75019700010617':['33071','33176','33662','34573'],
#                   'Copper_River_75019700007155':['33383','34186'],
#                   'Copper_River_75019700006876':['33071','34573'],
#                   'Copper_River_75019700012433':['33209','33534','33883','34829','35036'],
#                   'Copper_River_75019700010666':['34765'],
#                   'Copper_River_75003900065861':['29159','29720','30152'],
#                   'Copper_River_75003900049180':['29412','29492','29750','29839','30409','30599'],
#                   'Copper_River_75003900055856':['28905','29253','29951','30334'],
#                   'Copper_River_75003900084262':['28901','29105','29269','29408','29475','29614','29821','30165','30538'],
#                   'Prince_William_Sound_18357':['20697','21594','21911','22154'],
#                   'Prince_William_Sound_30774':['27499'],
#                   'Prince_William_Sound_41123':['26856','27269'],
#                   'Prince_William_Sound_41143':['28419'],
#                   'Prince_William_Sound_43513':['28851','30379'],
#                   'Prince_William_Sound_43793':['29114','29258','29829'],
#                   'Prince_William_Sound_44113':['29560','29832'],
#                   'Prince_William_Sound_45863':['28956','30138'],
#                   'Prince_William_Sound_46065':['26914','27003','27094','27171','27399','27731'],
#                   'Prince_William_Sound_62232':['28495','28728'],
#                   'Prince_William_Sound_66912':['27524','28198'],
#                   'Prince_William_Sound_72972':['27343','28560'],
#                   'Prince_William_Sound_82471':['27738','28161'],
#                   'Copper_River_75003900034359':['29613','30251'],
#                   }

# wrongWtd = {'Cook_Inlet_75004200001726':['Cook_Inlet_75004200002199','231-30-10080'],
#             'Cook_Inlet_75004200004105':['Cook_Inlet_75004200000559','226-50-16226'],
#             'Cook_Inlet_75004200001298':['Cook_Inlet_75004200010496','232-40-10190'],
#             'Cook_Inlet_75004200006640':['Cook_Inlet_75004200010672','242-41-10380'],
#             'Cook_Inlet_75004200009218':['Cook_Inlet_75004200000848','232-30-10247'],
#             'Cook_Inlet_75004200013577':['Cook_Inlet_75004200012465','232-23-10100'],
#             'Cook_Inlet_75004200000478':['Cook_Inlet_75004200005907','231-30-10160'],
#             'Cook_Inlet_75004200001167':['Cook_Inlet_75004200003332','232-10-10330'],
#             'Cook_Inlet_75004300007319':['Cook_Inlet_75004300003397','241-12-10100'],
#             'Cook_Inlet_75000400029059':['Cook_Inlet_75000400033841','247-50-10200-2081-3025'],
#             'Cook_Inlet_75000500006404':['Cook_Inlet_75000500002272','247-41-10200-2381-3161-4080-5001'],
#             'Cook_Inlet_75000300026273':['Cook_Inlet_75000300022808','247-41-10200-2370-3297'],
#             'Cook_Inlet_75000300000174':['Cook_Inlet_75000300021956','247-41-10200-2370'],
#             'Cook_Inlet_75000600015375':['Cook_Inlet_75000600040122','247-41-10200-2053-3220'],
#             'Cook_Inlet_75000600018321':['Cook_Inlet_75000600018460','247-41-10200-2053-3205-4099'],
#             'Cook_Inlet_75000600026866':['Cook_Inlet_75000600006507','247-41-10200-2053-3229-4200'],
#             'Cook_Inlet_75000600009692':['Cook_Inlet_75000600025058','247-41-10200-2053'],
#             'Cook_Inlet_75000200008552':['Cook_Inlet_75000200005953','247-41-10200-2130-3020'],
#             'Cook_Inlet_75000200002996':['Cook_Inlet_75000200018317','247-41-10200-2081-3100-4136'],
#             'Cook_Inlet_75000200012956':['Cook_Inlet_75000200000235','247-41-10200'],
#             'Cook_Inlet_75000200003487':['Cook_Inlet_75000200018781','247-41-10200'],
#             'Cook_Inlet_75000200003483':['Cook_Inlet_75000200008607','247-41-10200-2250'],
#             'Cook_Inlet_75005300003320':['Cook_Inlet_75005300007002','247-30-10090-2150'],
#             'Cook_Inlet_75005300018781':['Cook_Inlet_75005300022592','247-30-10090-2151'],
#             'Cook_Inlet_75005400022563':['Cook_Inlet_75005400034611','245-30-10084'],
#             'Cook_Inlet_75005400019698':['Cook_Inlet_75005400008084','243-50-10020'],
#             'Cook_Inlet_75005400000004':['Cook_Inlet_75005400029492','245-20-10230'],
#             'Copper_River_75019600174661':['Copper_River_75019600133175','212-20-10080-2401'],
#             'Copper_River_75019600198622':['Copper_River_75019600133538','212-20-10080-2331'],
#             'Copper_River_75003900024361':['Copper_River_75003900057807','212-20-10080-2100-3113'],
#             'Copper_River_75003900070258':['Copper_River_75003900034185','212-20-10080-2159-3021'],
#             'Copper_River_75003900008591':['Copper_River_75003900084672','212-20-10080-2100-3089'],
#             'Copper_River_75003900051393':['Copper_River_75003900084267','212-20-10080-2021'],
#             'Copper_River_75003900055093':['Copper_River_75003900023613','228-20-18420'],
#             'Copper_River_75003900054944':['Copper_River_75003900054448','228-70-10500'],
#             'Prince_William_Sound_18747':['Prince_William_Sound_18967','227-30-17780'],
#             'Prince_William_Sound_27856':['Prince_William_Sound_19416','226-40-16851'],
#             'Prince_William_Sound_30084':['Prince_William_Sound_29704','224-10-14540'],
#             'Prince_William_Sound_33106':['Prince_William_Sound_16467','226-50-16154'],
#             'Prince_William_Sound_42005':['Prince_William_Sound_39255','224-40-14904'],
#             'Prince_William_Sound_42595':['Prince_William_Sound_41025','224-40-14900'],
#             'Prince_William_Sound_43085':['Prince_William_Sound_40635','225-30-15070'],
#             'Prince_William_Sound_43255':['Prince_William_Sound_37935','226-20-16010'],
#             'Prince_William_Sound_44273':['Prince_William_Sound_35343','223-40-13390'],
#             'Prince_William_Sound_44605':['Prince_William_Sound_37955','226-20-16950'],
#             'Prince_William_Sound_45925':['Prince_William_Sound_28734','224-40-14860'],
#             'Prince_William_Sound_46613':['Prince_William_Sound_36553','224-10-14510'],
#             'Prince_William_Sound_46623':['Prince_William_Sound_34993','224-10-14500'],
#             'Prince_William_Sound_70762':['Prince_William_Sound_61812','222-10-12030'],
#             'Prince_William_Sound_75002':['Prince_William_Sound_60732','223-30-13180'],
#             'Prince_William_Sound_77992':['Prince_William_Sound_75842','224-10-14020'],
#             'Prince_William_Sound_80112':['Prince_William_Sound_78912','222-10-12040'],
#             'Prince_William_Sound_84641':['Prince_William_Sound_72242','222-20-12640'],
#             'Prince_William_Sound_89681':['Prince_William_Sound_75151','221-60-11520'],
#             } #
multirecslist

### Bristol Bay and Kodiak
use catid for Bristol Bay when re-assigning watersheds NOT the grid code.  This region was processed in Sub-watersheds so catid is the unique identifier.
Lower Weary and Snake Rivers are generally poorly routed near Dillingham.  Would need to force with stream burning here to get correct flow paths.  Synthetic Networks often routed over land in straight line paths and do not follow river channels.
Tunuing River also routed poorly, Synthetic network routes West to Ocean instead of into the Igushik River to the North.

### QC Dictionaries Created During First Review
#### Must use OIDS for BOTH MultiCode and Missed AWC records AND make sure you are using the correct version of the awcEventArcs and Lakes to pull OIDs from .  Check the input from this script and ensure that is the layer you have loaded in arcGIS while QC'ing ...Moron

In [3]:
# Dictionary to store catchments that were found with multiple AWC_CODE (different code not sp_ls) during spatial join.  Key is catid and val1 is primary awc record that should be associated with the catchment. THESE NEED TO BE ADDED TO THE ADDHUC12QC DICTIONARY ALONG WITH ASSOCIATED AWC OIDS
awcMultiCodes = {'Kodiak_75624': '259-10-10015', #Monashka Creek primary system in this catchment that continues to Monashka Bay
                 'Kodiak_75194':'252-38-10020', #Red Cloud River is primary system.
                 'Kodiak_63943':'252-34-10020-2019',
                 'Kodiak_49837':'257-50-10020',
                 'Kodiak_48997':'257-40-10028',
                 'Kodiak_47187':'258-40-10012',
                 'Kodiak_13958':'258-80-10140-2006',
                 'Kodiak_125845':'253-12-10020',
                 'Kodiak_125115':'259-41-10050',
                 'Kodiak_124945':'259-23-10010',
                 'Kodiak_124065':'259-24-10020',
                 'Kodiak_123415':'259-41-10040',
                 'Kodiak_120645':'254-10-10120',
                 'Kodiak_108606':'256-40-10010',
                 'Kodiak_107376':'256-20-10010', #Ayakulik River at confluence
                 'Kodiak_105086':'256-20-10010-2034',
                 'Kodiak_103196':'256-20-10010-2016',
                 'Kodiak_103016':'258-52-10040',
                 'Kodiak_102106':'256-30-10010',
                 'Kodiak_101146':'258-20-10069',
                 'Kodiak_100826':'255-10-10010-2059', #Misses Canyon Creek to the North
                 'Bristol_Bay_5038685':'326-00-10400', #Togiak, catchment and awc mis-alignment.  watershed good.
                 'Bristol_Bay_5034833':'326-00-10400-2108',
                 'Bristol_Bay_5034423':'326-00-10400-2081',
                 'Bristol_Bay_5033295':'326-30-10500',
                 'Bristol_Bay_5033213':'326-00-10400-2091',
                 'Bristol_Bay_5033104':'326-00-10400-2058',
                 'Bristol_Bay_5033043':'326-00-10400-2108-3028',
                 'Bristol_Bay_5030574':'326-10-10020',
                 'Bristol_Bay_5030514':'326-00-10400-2047-3021',
                 'Bristol_Bay_5021406':'326-10-10600-2006',
                 'Bristol_Bay_5019001':'326-00-10400-2189-3012',
                 'Bristol_Bay_4119952':'325-30-10100-2202',
                 'Bristol_Bay_4118552':'325-30-10100-2435',
                 'Bristol_Bay_4114362':'325-30-10100-2481'
                } #DON'T USE THIS ANYMORE.  ANYTHING YOU ARE THINKING ABOUT PUTTING IN HERE SHOULD GO IN THE addHuc12QcDict with OIDs from input awcEventsArc as values


# Double Check
addRev = ['Kodiak_66383',  # Small Inlet Huc with many AWC streams
          'Kodiak_63603',  # Network is strange and NHD poor but many recorded AWC streams in watershed that was constructed.
          'Kodiak_49707',  # Network is bad here, missing large trib because network routed into adjacent system instead of meeting with larger trib to the south near outlet.  Would need to force flow with stream burning to correct.  Recommend Dropping HUC 190207011001 due to poor synthetic network
          'Kodiak_125355', # Lower Sturgeon river Huc.  Strange small with watershed that almost exactly overlaps then watershed from the next upstream catchment (Kodiak_125245/ Huc 190207010813).  I think we should drop this one.
          'Kodiak_116535', # Spiridon Bay-Frontal Uyak Bay HUC 190207010607 - This Huc has only small segments of awc records that are already captured in other identified catchments.  Drop the Huc and this catchment will also be dropped.
          'Kodiak_108206', # This is the upper Karluk River HUC which has encompasses the large Karluk lake.  The awc line ends at the lake outlet outside of the catchment but still in the HUC and does not pass through the lake.  There are multiple additional awc records on tribs to the lake including the inlet.  Not sure which awc record to assign to the huc.  either outlet or inlet?
          'Kodiak_101736', # Lots of awc records converging at the bay but network doesn't connect (neither does NHD or new NHD from Kodiak)          '',#
          'Bristol_Bay_4097783', # A few tribs are missing but not sure if it is worth generating new watersheds to include them.  Would have to shift pretty far down and associate the correct awc record since it would no longer intersect the catchment
          'Bristol_Bay_4095504', #Headwaters Kokwok River, original outlet way too High in Catchment but network is messy here and synthetic stream diverts higher than it should, missing about halve of the watershed area in this HUC.  Drop?
          'Bristol_Bay_4094921', #Not sure if the trib directly to the West of this outlet should be included in the watershed area
          'Bristol_Bay_4116442', # Should this HUC include the Tikchik coming in on the east just on the edge of the HUC Boundary?  If not need to slide catchment us slightly to avoid the confluence
          'Bristol_Bay_4065317', # Supposed to be the outlet of the Tuking river but the synthetic network routed the outlet pretty far off and I would need to force the flow by burning the stream in to correct this. #Need to bookmark this on the main map. - I think this can be approximated to the outlet Identified for HUC 190303060402 as that is where the synthetic approximation of the Turing River outlet would be and this HUC was
          'Bristol_Bay_4063777', #Synthetic network is highly shifted for the lower snake river. Would need to burn this in to force flow. Lots of artifical 1st orders created here
          'Bristol_Bay_4033648', #Tunuing River, outlet of this river incorrectly routed to the West instead of joining Igushik to the North
          'Bristol_Bay_2085476', #Trib to Alagnak River, Network routed incorrectly here and diverts North at HUC bound instead of flowing south to actual confluence point
          ]

#Drop large coastal HUCs and possibly some inlets as they do not fit the scope of the project.
DropHucs = ['190207011108',
            '190207011206',
            '190207010708',
            '190207011905',
            '190207010814',
            '190207011410',
            '190207011506',
            '190207010208',
            '190303060803',
            '190303051505',
            '190303051103',
            '190303051907',
            '190303052002',
            '190302062311',
            '190207010813', #Lower Sturgeon River - See notes above 'Kodiak_125355' will be dropped but contributing Hucs basically start at the outlet and capture everything this would have.
            '190207010607', #Spiridon Bay-Frontal Uyak Bay HUC 190207010607 - see above
            '190207010707', #Zachar Bay, no AWC Records that are not already captured by a contributing HUC present.  Drop this coastal HUC
            '190303051103', #Togiak Bay HUC - nothing really in this HUC that I think would qualify, Bristol_Bay_5033175
            '190303051803', #Bristol_Bay_5019146, catchment intersects the main stem of Kanik River AWC record, no actual awc records in this huc
            '190303051805', #Kulukak Bay, no awc records in this Huc that are not already captured at a contributing HUC, no records at all in the watershed that was constructed
            '190303051907', #Tvativak Bay, no awc records in this Huc that are not already captured at a contributing HUC, no records at all in the watershed that was constructed
            '190303052002', #Coastal Huc
            '190303051505', #Coastal Huc
            '190303011303', #Lake Huc
            '190207011001', #Network is Bad here and misses about half of the watershed
            '190303010104', #Woods creek in Bbay, awc from Nush crosses woods creek huc, no records on woods creek
            '190303011002', #unnamed trib to unnamed trib with awc record where intersection occurs, no actual records within this Huc
            '190303011006', #unnamed trib to harris creek, outlet of this trib intersects with the harris creek record, no actual records within this HUC
            '190303030103', #no records in this huc, intersects with Nunachuak Creek at confluence
            '190303020501', #no records in this huc, intersects with Little Mulchatna River at confluence
            '190303030204', #trib to Tunravik Creek, no records in this Huc, outlet intersects at confluence
            '190303060402', #no records in this huc, Tunravik River records slightly intersects huc boundary.
            '190303030603', #no records in this huc, unnamed system records slightly intersects huc boundary.
            '190303020103', #Huc has no records, outlet that was identified is actually on the Mulchatna.
            '190303060403', #no records in this HUC, intersects Tuklung at N bound
            '190303060402', #no records in this HUC, intersects Tuklung at E bound
            '190303060803', #Coastal Huc
            '190302040603', #no recs in this HUC, intersects with records of small side channel? on Savanoski River
            '190302040207', #No recs in this HUC, intersects American Creek on boundary
            '190302041104', #No recs in this HUC, intersects with recs on boundary
            '190302050606', #No recs in this HUC, intersects with recs on boundary
            '190302050502', #No recs in this HUC, intersects with recs on boundary
            '190302051101', #No recs in this HUC, intersects with recs on boundary
            '190302051102', #
            '190302051103', ##No recs in this HUC, intersects with Chulitna recs at confluence
            '190302062311', #Kvichak Bay
            '190302061502', #No recs in this HUC, intersects with recs on boundary
            '190302050103', #Glacier Fork Tlikakila, no recs in this huc, intersect with main stem Tlika at confluence
            '190302031110', #Egegik River, synthetic outlet id'd in Outlet Egegik River HUC
            '190302030603', #No recs in this HUC, intersects with recs on boundary
            '190302031107', #No recs in this HUC, intersects with recs on boundary
            ]


# Dictionary to relate catchments to awc using awc OIDs for those missed in spatial join AND those with multiple awc records, watershed is correct but awc line did not intersect outlet catchment.
# MAKE SURE AWC DATA LOADED IN PRO IS THE SAME REFERENCED AS THE INPUTS FOR THIS SCRIPT. OIDS FROM PREVIOUS VERSIONS OF THE SAME RECORDS WILL NOT MATCH AND YOU WILL HAVE TO DO THIS ALL OVER AGAIN WHEN YOU FINALLY REALIZE AFTER HAVING ENTERED A FEW 100 OF THESE
addHuc12QcDict = {'Kodiak_80022':['10688','11472','11883','12600','12893'],
                  'Kodiak_79642':['12344','12456','12540','12829'], #Bookmarked as long lagoon
                  'Kodiak_63963':['5006','5875','6287'],
                  'Kodiak_49617':['2136','2348','2586','2989','3583','3931','4384'], #Dog Salmon Creek, awc polyline just outside of catchment
                  'Kodiak_47207':['3973','4185','4351'], #Newman Bay-Frontal Sitkalidak Strait, awc polyline starts further up the synthetic network
                  'Kodiak_46167':['2608','3354'], #awc polyline just outside of catchment
                  'Kodiak_14958':['3089','3304','3759','4031'],#awc polyline just outside of catchment
                  'Kodiak_14858':['2123'], #synthetic network slightly south of awc path, watershed looks good
                  'Kodiak_131775':['4551','5450','5935','6071'], #watershed looks good,
                  'Kodiak_130995':['4440','4741','5021','5247','5498','5634','6055'], #Karluk River, slight mismatch between awc and synthetic network, watershed looks good
                  'Kodiak_125235':['4645','5749','5981'], # Watershed looks good, awc record starts slight further upstream outside of catchment
                  'Kodiak_124215':['4402','4448','4790','5028','5255','5346'], # Miam River, Watershed looks good, awc line slightly outside of catchment
                  'Kodiak_122785':['5261','5420'],
                  'Kodiak_122415':['4664','4801'], # South Arm Uganik
                  'Kodiak_104296':['5155','5338','5731','5963'], # awc line just outside catchment, watershed looks good
                  'Kodiak_103266':['2161','2262','2828','3498'], # awc line outside of catchment and network error misses large trib to the East
                  'Kodiak_101826':['5041','5357','5452','6165'], # Watershed here is missing a large fork that was interpreted as its own stream in the synthetic network
                  'Kodiak_46347':['4470','4545','4841','5276','5855','6084'], #Alpine Creek slightly outside of catchment
                  'Kodiak_75624':['4601','5109','5156','6352'], #Monashaka Creek, Synthetic network misses Virginia Creek Confluence to the WSW
                  'Kodiak_75194':['5233','5949','6382'], #Multiple AWC Records, Red Cloud River is the main system
                  'Kodiak_63943':['4842','4936'], #Watershed looks good
                  'Kodiak_48997':['4743','5919'], #Multiple records here, Talifson's Creek main system
                  'Kodiak_47187':['4973','5297','5435','5480','5618'], #Multiple records here, Ocean Beach Creek main system
                  'Kodiak_13958':['2855','3042'], #Multiple records here, using OIDs from segment downstream from the fork
                  'Kodiak_125115':['4482','4490','4580','5067','5479','5779'], #Multiple records here, Saltery Creek is the main system here
                  'Kodiak_124945':['4494','4677','5491','5810'], #Multiple records here, American River is the main system here
                  'Kodiak_124065':['4667','4964','5378','6101'], #Multiple records here, Olds River is the main system in this HUC
                  'Kodiak_123415':['4509','5288','5610','6095'], #Multiple records here , Hurst Creek is the main system in this HUC
                  'Kodiak_120645':['4646'], #Multiple records here, dropping this coastal HUC entirely
                  'Kodiak_108606':['2557','2934','3096','3551','4140'], #Sturgeon River is the main system in this HUC
                  'Kodiak_107376':['2297','2966'], #Multi recs here, Ayakulik is main system
                  'Kodiak_105086':['6139'], #Multi recs
                  'Kodiak_103196':['4620','4782','4875','5564','5936'], #Multi recs, trib to the Ayakulik
                  'Kodiak_103016':['4406','4906','5462','5993'], # Multiple recs here
                  'Kodiak_102106':['2606','2663','3165','3409','3515','3603','4163'], #multiple recs here
                  'Kodiak_101146':['5558','6272','6300'], #multiple records here
                  'Kodiak_100826':['4506','5152','5728','5960'], #Multi recs, O'Malley River main system
                  'Bristol_Bay_5038685':['10998','11353','11377','11879','11903','12020','12395','12912','13026'] , #Togiak River, Waterhsed looks good, awc record just outside catchment
                  'Bristol_Bay_5021956':['10704','11198','11547','12217','12415','12817'],#Catchment and watershed good, awc line starts further upstream on Osviak River
                  'Bristol_Bay_4112382':['18192'], #Moose Creek, Synthetic network begins slighty higher up the HUC than NHD/AWC records.
                  'Bristol_Bay_5034833':['18220','18269','18730','19000','19111','19682','20333'], #Ongivinuck River main system in this HUC
                  'Bristol_Bay_5034423':['18428','18889','19573','19738','19839','20303'], #Kashaiak River, mult recs
                  'Bristol_Bay_5033295':['10704','11198','11547','12217','12415','12817'], #Osviak River, mult recs
                  'Bristol_Bay_5033213':['18139','18198','18657','18867','20389'], #Kemuck River, mult recs
                  'Bristol_Bay_5033104':['18507','18811','19215','19382','19954','20412'], #Pungokepuk Creek, multi recs
                  'Bristol_Bay_5033043':['18346','19018','19188','20430'], #mult recs, No Lake Creek
                  'Bristol_Bay_5030574':['11361','11887','12203','12264','12782','12864'], #Kulukak River, mult recs
                  'Bristol_Bay_5030514':['18764'], #Catchment overlaps Gechiak Creek as well as main awc record in Huc
                  'Bristol_Bay_5021406':['11530'], #Catchment overlaps Negukthlik River, Ungalikthluk River (name not recorded in AWC) is main sys
                  'Bristol_Bay_5019001':['18158','18502'], #Catchment intersect record for Izavieknik River
                  'Bristol_Bay_4114362':['25023','26231','26613'], #Klutapuk Creek, catchment slighty intersects the Nushagak record
                  'Bristol_Bay_4114302':['18408','18658','19640','19933','20446'], #Mosquito River at the Mulchatna, slightly intersect Mulch
                  'Bristol_Bay_4111042':['18503'], #Unnamed trib that intersect King Salmon River awc, watershed looks good
                  'Bristol_Bay_4110612':['26143'], #Unnamed trib that intersect King Salmon River awc, watershed looks good
                  'Bristol_Bay_4110112':['25336'], #Unnamed trib that intersect Klutapuk Creek awc, watershed looks good
                  'Bristol_Bay_4107154':['18327','18891','19113','19384','19463','19495','20247','20305'], #Nuyakuk River at Nushagak, watershed pretty good, missing small trib
                  'Bristol_Bay_4106774':['19163','19623'],#unnamed trib to the Mulchatna that intersect mulch record too
                  'Bristol_Bay_4106762':['19551'], #Vukpalik Creek slightly overlaps awc record at confluence with another trib
                  'Bristol_Bay_4105772':['25744','26127'], #Unnamed trib to McGeary Creek, slightly overlaps
                  'Bristol_Bay_4103902':['18377','18377'],#trib to Vukpalik Creek, slightly overlaps
                  'Bristol_Bay_4101893':['18436','18876','19155','19340','19390','19920','19941','19998','20290'], #Mulchatna River, record just outside catchment
                  'Bristol_Bay_4099614':['18430','18509','18732','18834','20070','20145'], #Napotoli Creek, slightly overlaps another trib
                  'Bristol_Bay_4099464':['18266','18560','19212','19400','19679','20029','20140'], #unnamed trib to Nuyakuk, overlaps main stem
                  'Bristol_Bay_4099444':['18903','18936'], ##unnamed trib to Napotoli, overlaps main stem
                  'Bristol_Bay_4098683':['20160'], # AWC record just starts a bit upstream from outlet catchment, outside of catchment
                  'Bristol_Bay_4098534':['18185','18752','19020','19240','19298','19595'], #Stuyahok, overlaps small trib coming in downstream
                  'Bristol_Bay_4098514':['18367','18469','19038','19422','19802','19917'], #Unnamed trib to Kokwok River, overlaps kokwok
                  'Bristol_Bay_4097941':['18436','18876','19155','19340','19390','19920','19941','19998','20290'], #Section of Mulchatna, moved catchment down
                  'Bristol_Bay_4097764':['18925'], #small trib to/overlaps stuyahok
                  'Bristol_Bay_4097543':['18178','18285','18556','18637','19588','19698','20049','20082'], #North Fork Koktuli, overlaps tribs
                  'Bristol_Bay_4119972':['18436','18876','19155','19340','19390','19920','19941','19998','20290'], #another section of mulchatna
                  'Bristol_Bay_4097174':['20060'], #network joins Nushagak slightly higher than NHD and Awc, awc record slightly SW of catchment
                  'Bristol_Bay_4097073':['18479','18593'], #Multiple records, catchment slightly intersect Koktuli record at confluence
                  'Bristol_Bay_4096733':['18391','18581','18640','19235','20187'], #Outlet for Arrow Creek intersect Nuyakuk at confluence
                  'Bristol_Bay_4095964':['18719','18792','20132','20233'], #Trib outlet intersects Stuyahok at confluence
                  'Bristol_Bay_4094641':['24834','25441','25797','25831','26121','26585','26649'], #Nikadavna Creek outlet intersects Chilchitna at confluence
                  'Bristol_Bay_4093591':['26099'], #Polly creek outlet overlaps Nush at confluence
                  'Bristol_Bay_4092891':['10813','11445','11917','12521','12718','12727','12810','12926'], #Record just outside catchment, Nushagak
                  'Bristol_Bay_4092121':['26689'], #Crooked Creek outlet overlaps nush record
                  'Bristol_Bay_4091781':['24669','25409','25857','26020','26323','26708'], #Unnamed trib to Chichitnok, overlaps at confluence.
                  'Bristol_Bay_4089713':['18242','19784','20335'], #Harris Creek at Nushagak confluence, outlet intersect with Nush too
                  'Bristol_Bay_4089671':['26654'], #Trib to EF Mosquito, intersects at confluence
                  'Bristol_Bay_4088865':['18436','18876','19155','19340','19390','19920','19941','19998','20290'], #Mulchatna at confluence with Nush, synthetic
                  'Bristol_Bay_4118552':['18362','18387','18668','19201','19797','19912','20048','20156'], #King Salmon River, catchment slightly intersects trib coming in downstream
                  'Bristol_Bay_4105244':['18185','18348','18450','18752','19020','19240','19298','19595'], #Stuyahok catchment intersects Mulchatna
                  'Bristol_Bay_4106152':['20095'], #Wrong Awc record was associated with this catchment, actual record just outside catchment to the West
                  'Bristol_Bay_4087083':['18446','20064'], #outlet catchment intersects another record at confluence
                  'Bristol_Bay_4087071':['24918','26790'], #outlet catchment intersects Chilikadrotna record at confluence
                  'Bristol_Bay_4086803':['18218','19052','19459','19659','19952','20008'], #Trib to Swan river, outlet catchment intersects Swan Record
                  'Bristol_Bay_4086795':['18315','18418','18879','19372','19729','19829','19950'], #Section of Kokwok that intersects trib with records.
                  'Bristol_Bay_4086751':['24512','25468'], #Dummy creek at Chilchitna, outlet intersects Chilchitna records
                  'Bristol_Bay_4085381':['20203'], #Mosquito river at confluence, catchment interscets records on trib
                  'Bristol_Bay_4085325':['18253','19060','19365','19578'], #Unnamed trib to Kokwok, catchment intersects at confluence
                  'Bristol_Bay_4084971':['24591'], #Burnt Cabin Creek at Nushagak, outlet intersects at confluence
                  'Bristol_Bay_4084941':['25879','25936','26730'], # Unnamed trib that intersects with Chichitnok River records at confluence
                  'Bristol_Bay_4084581':['24490','24781','25219'], # Unnamed trib that intersects with Nushagak records at confluence
                  'Bristol_Bay_4084315':['18130','18968','19077','19243','19541','20458'], #Kenakuchuk Creek outlet intersects with Kokwok records at confluence
                  'Bristol_Bay_4083471':['25716'], #Unnamed trib that intersects with Chichitnok at confluence
                  'Bristol_Bay_4082741':['26766'], #unnamed trib that intersects with Mosquito River at confluence
                  'Bristol_Bay_4080805':['19107','19779'], #unnamed trib that intersects with Kokwok at confluence
                  'Bristol_Bay_4080635':['19731'], #unnamed trib that intersects with Nunachuak at confluence
                  'Bristol_Bay_4080545':['18727','18907','19972'], #unnamed trib that intersects with Klutuk Creek at confluence
                  'Bristol_Bay_4076505':['19499','19742','20129','20309'], #fenno creek, record just outside catchment
                  'Bristol_Bay_4075046':['10752','12084','12378','12871'], #Ongoke River catchment intersects records of trib at confluence
                  'Bristol_Bay_4072656':['12497','13154'], #outlet stream of Ualik Lake that intersects Ongoke River records at confluence
                  'Bristol_Bay_4068046':['10669'], #intersects with trib to lower klutuk to the North
                  'Bristol_Bay_4067386':['10827'], #trib that intersects with Muklung River records at confluence
                  'Bristol_Bay_4066846':['11360','12781'], #longarm creek record just outside outlet
                  'Bristol_Bay_4066507':['10782','10813','11445','11711','11917','12324','12521','12718','12727','12810'], #Nushagak near outlet, missing Akomilinak Creek due to network routing but sliding down far enough to include would also add the Wood River
                  'Bristol_Bay_4065337':['10919','11035','11622','11737','12055','12832'], #Igushik River, outlet intersects Tunuing River records
                  'Bristol_Bay_4063687':['12393'], #Portage Creek, overlaps main system at confluence
                  'Bristol_Bay_4063647':['11072','11191','11424','12035','12091','12210','12694'], #Snake River, new outlet shifted slighty ds to include missing trib but records are outside catchment bound
                  'Bristol_Bay_4063447':['11393','11474','11483','11589','11968','12295','12489','12602','13092'], #Iowithla River, outlet catch intersects Nush records at confluence
                  'Bristol_Bay_4062937':['12504'], #Unnamed trib to the Nush, overlaps with records in side channel.
                  'Bristol_Bay_4055500':['24433'], #New watershed catchment that will intersect Multchatna record.
                  'Bristol_Bay_4084501':['25167'], #New watershed that will intersect main system at confluence, original watershed incorrectly included the main system.
                  'Bristol_Bay_4051080':['18876','19155','19340','19390','19726','19920','19941','19998'], #Mulchatna River, outlet catchment very small and records just outside boundary.
                  'Bristol_Bay_4050550':['25551'], #Trib to Bonanza, outlet intersects records at confluence
                  'Bristol_Bay_4050230':['24982','26325'], #Summit Creek at Mulchatna, outlet intersects Mulchatna recs
                  'Bristol_Bay_4034378':['10919','11035','11622','11737','12055','12832'], #Outlet Igushik River, record just outside catchment bound
                  'Bristol_Bay_4033648':['110133','11306'], ##Tunuing River, outlet of this river incorrectly routed to the West instead of joining Igushik to the North. Catchment that was identified west of actual record due to synthetic system error.
                  'Bristol_Bay_3034116':['10881'], #Ukak River, record just outside catchment bound
                  'Bristol_Bay_3032286':['11436','12128','12419','12647','12708','13136'], #multiple recs at confluence
                  'Bristol_Bay_3024783':['10781','10895','11246','11390','11423','11736','12115','12149','12435'], #Outlet Naknek, record starts slight US of catchment
                  'Bristol_Bay_3024213':['10804','11298'], #Smelt Creek at Naknek, catchment intersects Naknek recs
                  'Bristol_Bay_3023733':['10775','11500','11909','12285','12803','12919'], #Pauls Creek at confluence with Naknek, catchment intersects Nak recs.
                  'Bristol_Bay_3021962':['10719','12034'], #Headwaters King Salmon Creek, overlaps recs of rib to the N at confluence
                  'Bristol_Bay_3021744':['12802'], #Gorge creek at confluence with Hardscrabble Creek, mult recs
                  'Bristol_Bay_3016301':['11621','12925'], #trib to Pauls creek intersects recs at confluence
                  'Bristol_Bay_3006179':['11282','12878'], #trib to Big creek, mult recs at confluence
                  'Bristol_Bay_2088616':['11056','11291','12481','12854'], #Ben Courtney Creek, mult recs at confluence with small trib
                  'Bristol_Bay_2087446':['11071','12090','12209','13099'], #trib with mult recs at confluence with another unnamed trib
                  'Bristol_Bay_2086386':['11218','12723','13047'], #Moraine Creek, record just outside catchment bound
                  'Bristol_Bay_2083946':['19064'], #trib to Kaskanak Creek, multiple records intersect catchment
                  'Bristol_Bay_2082873':['19574'], #trib to Chekok intersects recs at confluence
                  'Bristol_Bay_2082862':['19774'], #Kijik River, mult recs in this area
                  'Bristol_Bay_2082856':['18766','19333','19362','19552','19719'], #Pecks Creek, outlet intersects Kvichak recs
                  'Bristol_Bay_2082586':['18236','18444','19753','19778','19970','20051'], #Ole Creek, outlet intersects Kvichak recs
                  'Bristol_Bay_2081436':['11136','12526','12640','12931'], #Funnel Creek catchment intersect Moraine Cr at confluence
                  'Bristol_Bay_2080866':['10812','11190','12870'], #Catchment in flat area and extends through records of creek to the E
                  'Bristol_Bay_2076254':['18249','19055','19662'], #unnamed trib to Illiamna lake near Chekok, catchment small and awc record outside bounds
                  'Bristol_Bay_2075404':['19739','20304'], #Pile River, Multiple recs here in what looks like a side channel
                  'Bristol_Bay_2074724':['18890','19132','19974'], #Trib to upper Talarik Creek, small catchment and record begins slightly US out of catchment boundary
                  'Bristol_Bay_2070527':['11019','11514','11628','11743','12156','12527','12838','13014'], #HUC above Kvichak Bay/outlet, records just outside catch bound
                  'Bristol_Bay_2070127':['10695','10759','11400','11680','11894','12788','12903'], #Alagnak at Kvichak, outlet intersects Kvichak recs at confluence
                  'Bristol_Bay_2069504':['18180','18558','18820','18905','19283','19509','20319','20341'], #Kaskanak Cr, outlet intersects small trib with recs to the N
                  'Bristol_Bay_2069334':['18979','19934','20178','20283'], #Lower Talarik Cr, multiple recs here
                  'Bristol_Bay_2069144':['20163','18854','20268'], #Chinkeyles Cr. catch intersects recs on Illiamna R. at confluence
                  'Bristol_Bay_2068895':['18180','18558','18905','19283','19509','20319','20341'], #Kaskanak Cr. catchment intersects records on trib to the W
                  'Bristol_Bay_2068715':['19751'], #Trib to Kaskanak, mult recs at confluence
                  'Bristol_Bay_2068474':['19819','19934','20283'], #Trib to Low. Talarik Cr., mult recs at confluence
                  'Bristol_Bay_2067465':['18180','18558','18905','19283','19509','20319','20341'], #Kaskanak Cr. mult recs at where small trib joins to the W
                  'Bristol_Bay_2066734':['18353'], #Trib to Illiamna R. mult recs at confluence
                  'Bristol_Bay_2064385':['19393'], #Trib to Ben Courtney Cr. mult recs at confluence
                  'Bristol_Bay_2064245':['18631','18951'], #Trib to Gibralter Cr. mult recs at confluence
                  'Bristol_Bay_2063257':['10863'], #Trib to Kulik Lake, mult recs
                  'Bristol_Bay_2042341':['26637'], #Chokotonk River, small catchment and record located outside boundary
                  'Bristol_Bay_1026613':['11253','11359','12122','12757','13130'], #Takayofo Cr, mult tribs and recs in this area at confluence with King Salmon River
                  'Bristol_Bay_1026463':['2217','2483','2563','2671'], #Unnamed trib to Egegik with mult recs at confluence
                  'Bristol_Bay_1025783':['12077','12969'], #rec starts outside bound of outlet, Upper Swampy River
                  'Bristol_Bay_1025113':['2899','3352'], #Trib to Takayofo Cr, mult tribs and recs in this area at confluence with King Salmon River
                  'Bristol_Bay_1029732':['11176','11994','12309','12594'], #Angle Creek at confluece with Takayofo Cr, mult recs
                  'Bristol_Bay_1021914':['3298','3753'], #Kejulik River, outlet catchment very small (single cell) and record just outside bound
                  'Bristol_Bay_1016515':['2192','3126'], #East Fork Kejulik River, mult recs here
                  'Bristol_Bay_1016325':['2249','3298','3753'], #Kejulik River mainstem, mult recs here
                  'Bristol_Bay_1015685':['4190'], #Mult recs in this area
                  'Bristol_Bay_1015186':['2249','3298','3753'], #Kejulik River mainstem near Katrine Creek confluence, mult recs here
                  'Bristol_Bay_1014821':['10951','11131','11213','11560','12150','13063'], #Mainstem King Salmon River, mult recs where small trib joins to the SW
                  }

# Dictionary to relate huc 12 outlets without awc data to correct catchment/awc - new watersheds/covariates need to be calculated
wrongWtd = {'Kodiak_79862':['Kodiak_78362','252-33-10020'],
            'Kodiak_79034':['Kodiak_75664','253-11-10040'],
            'Kodiak_77952':['Kodiak_75052','251-82-10029'],
            'Kodiak_79842':['Kodiak_79862','252-33-10010'],
            'Kodiak_47797':['Kodiak_47317','258-54-10012'], # Shifted to Catchment with most AWC records
            'Kodiak_49837':['Kodiak_46347','257-50-10030'], # Shifted to a smaller system in the HUC.  Originally identified as the catchment for Deadman River but the network there is not accurate and would need stream burning to force a correction.
            'Kodiak_103116':['Kodiak_99996','254-20-10010'], # Shifting across the bay to the largest AWC line in HUC.  Watershed that was created almost exactly the same as contributing HUC
            'Bristol_Bay_5029122':['Bristol_Bay_5029102','326-00-10400-2091-3201'], # Need to go upstream one catchment to avoid including the fork from the adjoining HUC to the East.
            'Bristol_Bay_5028612':['Bristol_Bay_5028672','326-00-10400-2163'], # Shifting to the next downstream catchment to capture small trib
            'Bristol_Bay_5026052':['Bristol_Bay_5026192','326-00-10400-2178'], # Shifting to the next downstream catchment to capture small trib
            'Bristol_Bay_4120132':['Bristol_Bay_4120152','325-30-10100-2202'], # Shifting downstream to include missed channel of mulchatna
            'Bristol_Bay_4116442':['Bristol_Bay_4116432','325-30-10100-2249-3142-4163'],
            'Bristol_Bay_4107454':['Bristol_Bay_4105234','325-30-10100-2202-3044'], # Stuyahock, wrong catchment
            'Bristol_Bay_4105964':['Bristol_Bay_4099714','325-30-10100-2202-3080'], # Koktuli, wrong catchment was identified for this HUC
            'Bristol_Bay_4102223':['Bristol_Bay_4093963','325-30-10100-2202-3125'], # Dixie Creek, Had to go upstream to actual synthetic approximation of outlet
            'Bristol_Bay_4100083':['Bristol_Bay_4068633','325-30-10100-2202-3076'], # Pike Creek Huc, shifted catchment to synthetic approx of Pike creek outlet
            'Bristol_Bay_4099023':['Bristol_Bay_4093813','325-30-10100-2202-3080-4029-5039-6045'],
            'Bristol_Bay_4098664':['Bristol_Bay_4098614','325-30-10100-2202-3080'], # Shifted slightly upstream to avoid including trib that should not have been included here
            'Bristol_Bay_4098081':['Bristol_Bay_4091891','325-30-10100-2202-3420'], # Shifting to appropriate catchment for this HUC
            'Bristol_Bay_4097981':['Bristol_Bay_4095631','325-30-10100-2202-3446'], # Shifting upstream to avoid including the Mulchatna
            'Bristol_Bay_4097911':['Bristol_Bay_4097941','325-30-10100-2202'], # Shifting downstream to include portion of the Mulchatna that was missed, need to establish oid relationship because new outlet does not intersect awc line
            'Kodiak_125845':['Kodiak_126005','253-12-10020'], #Network is messy here, shifting downstream to where the synthetic network approximation of the uganik mergers with the rest of the watershed.
            'Bristol_Bay_4119952':['Bristol_Bay_4119972','325-30-10100-2202'], #Mulchatna catchment that needs to be moved downstream slightly, OID relation also needs to be established because outlet does not intersect
            'Bristol_Bay_4096891':['Bristol_Bay_4096991','325-30-10100-2202-3510'], #Shifting downstream to include some tribs that come in lower
            'Bristol_Bay_4095641':['Bristol_Bay_4055540','325-30-10100-2202'], #Need to shift upstream to avoid including Big Bonanza Creek.
            'Bristol_Bay_4094233':['Bristol_Bay_4090253','325-30-10100-2202-3080-4058'], #Need to slide outlet catchment upstream to avoid capturing trib to the West.
            'Bristol_Bay_4093891':['Bristol_Bay_4093871','325-30-10100'], #Nush near confluence with fifteen mile creek, need to slide upstream to avoid including 15mile.
            'Bristol_Bay_4090071':['Bristol_Bay_4088891','325-30-10100-2202-3510-4089'], #Section of Ptarmigan creek, outlet needs to shift upstream to avoid trib to the SE
            'Bristol_Bay_4090011':['Bristol_Bay_4089921','325-30-10100'], #Portion of Nushagak, need to slide catchment upstream to avoid trib to the N
            'Bristol_Bay_4108222':['Bristol_Bay_4108242','325-30-10100-2299'], #Klutuspak Creek, shifting ds one catchment to include missed trib
            'Bristol_Bay_4107994':['Bristol_Bay_4108014','325-30-10100'], #Section of Nush, need to slide catchment down to include missed trib
            'Bristol_Bay_4107732':['Bristol_Bay_4090833','325-30-10100-2311'], #Shifting ds slightly to include missing trib
            'Bristol_Bay_4083715':['Bristol_Bay_4083725','325-30-10100-2129-3040'], #Kenakuchuk Creek, shifting ds one catchment to include missing trib
            'Bristol_Bay_4076065':['Bristol_Bay_4076505','325-30-10100-2031-3118-4006'], #Fenno Creek, sliding ds to include missing trib
            'Bristol_Bay_4073646':['Bristol_Bay_4073516','325-30-10100-2031-3028'], #Muklung River , sliding catchment up to avoid trib that was misrouted in synthetic network
            'Bristol_Bay_4071596':['Bristol_Bay_4071606','325-20-10030-2009'], #Weary River, sliding ds slightly to include missed trib
            'Bristol_Bay_4068006':['Bristol_Bay_4068046','325-30-10100-2112'], #lower klutuk creek, intersects trib at confluence
            'Bristol_Bay_4066027':['Bristol_Bay_4066047','325-30-10100'], #Nushagak at large fork, sliding ds slightly to include fork
            'Bristol_Bay_4065827':['Bristol_Bay_4065837','325-30-10100'], #Nushagak at Keefer Cutoff fork, sliding ds slightly to include fork
            'Bristol_Bay_4065317':['Bristol_Bay_4032098','325-10-10010-2023'], #Tuklung River, Network is bad here, moving upstream to approximate location of where the tuklung was rerouted.  Leaving as is just builds off the Nushagak.  Stream burning would be necessary to force the correct path
            'Bristol_Bay_4063637':['Bristol_Bay_4063647','325-20-10030'], #Snake River, network not great here, shifting ds slightly to include missed tribs
            'Bristol_Bay_4059867':['Bristol_Bay_4071786','325-30-10100-2110-3115-4403'], #Record for this Huc not on the outlet that was identified
            'Bristol_Bay_4055510':['Bristol_Bay_4055500','325-30-10100-2202-3545'], #Shifting upstream to avoid including the Mulchatna
            'Bristol_Bay_4054660':['Bristol_Bay_4084501','325-30-10100-2497-3226-4080'], #small trib to unnamed system that incorrectly includes main system in watershed area. Shifting slightly upstream above confluence
            'Bristol_Bay_3033946':['Bristol_Bay_3033956','324-20-10140-2200'], #Outlet for Margot Creek, shifting ds slightly to include missed tribs
            'Bristol_Bay_3033656':['Bristol_Bay_3033806','324-20-10140-2194-3039'], #Brooks River, shifting ds slightly to include missed trib
            'Bristol_Bay_3031465':['Bristol_Bay_3031545','324-20-10140-2245'], #Savonski River, outlet for American Creek identified, this should be included in watershed area but need to shift ds to get actual Savonoski River outlet.
            'Bristol_Bay_3024543':['Bristol_Bay_3024533','324-20-10140'], #Naknek at confluence with Pauls Creek, shifting slightly us to remove pauls creek from watershed
            'Bristol_Bay_3019031':['Bristol_Bay_3018051','324-20-10140-2245-3029-4113-5011'], #wrong outlet identified for this HUC, shifting US
            'Bristol_Bay_2089766':['Bristol_Bay_2089766','324-10-10150-2053'], # Trib to Kvichak, watershed erroneously included Kvichak, shifting us to remove.
            'Bristol_Bay_2088346':['Bristol_Bay_2069835','324-10-10150-2097'], #Headwaters Ben Courtney, need to shift upstream slightly to avoid trib at confluence
            'Bristol_Bay_2085482':['Bristol_Bay_2084502','324-10-10150-2207-3173-4050'], #Koksetna River, incorrect outlet identified, shifting US to actual outlet
            'Bristol_Bay_2085476':['Bristol_Bay_2068087','324-10-10150-2010-3035'], #Trib to Alagnak River, Network routed incorrectly here and diverts North at HUC bound instead of flowing south to actual confluence point. Shifting to confluence outlet but this will leave out a large portion of the headwaters
            'Bristol_Bay_2084722':['Bristol_Bay_2083292','324-10-10150-2207-3301'], #Tlikakila River, wrong outlet catchment ID'd, shifting US to avoid including Chokotonk in watershed
            'Bristol_Bay_2081566':['Bristol_Bay_2081546','324-10-10150-2136'], #Pecks Creek outlet, shifting us to avoid including unnamed trib to the S that has its own HUC
            'Bristol_Bay_2079192':['Bristol_Bay_2079202','324-10-10150-2207-3214'], #Currant Creek, shifting ds one catchment to capture missed trib and awc recs
            'Bristol_Bay_2074484':['Bristol_Bay_2072044','324-10-10150-2267'], #Chekok Creek near Canyon Confluence, routed poorly here so shifting upstream to avoid including Canyon, this shift will exclude a portion of the watershed and associated tribs
            'Bristol_Bay_2074434':['Bristol_Bay_2074294','324-10-10150-2273'], #Canyon Creek near Chekok Confluence, routed poorly here so shifting upstream to avoid including Chekok
            'Bristol_Bay_2073204':['Bristol_Bay_2073174','324-10-10150-2402-3025'], #Trib to Illiamna river, shifting US to avoid including Illiamna R
            'Bristol_Bay_2070037':['Bristol_Bay_2064327','324-10-10150-2010-3030'], #Trib to Alagnak, shifting US to remove Alagnak from watershed area
            'Bristol_Bay_2070227':['Bristol_Bay_2070217','324-10-10150'], #Kvichak at Alagnak confluence, shifting US to avoid including Alagnak
            'Bristol_Bay_2040601':['Bristol_Bay_2040341','324-10-10150-2207-3301'], #Tlikakila River, shifting US slightly to avoid including trib to the N
            'Bristol_Bay_1030062':['Bristol_Bay_1030072','322-10-10080-2016'], #Swampy River, shifting DS slightly to include missed tribs
            'Bristol_Bay_1022284':['Bristol_Bay_1022264','322-10-10080-2058'], #Shifting slightly US to avoid trib to the W
            'Bristol_Bay_1006858':['Bristol_Bay_1006868','322-10-10080-2292'], #Burls Creek , Shifting DS slightly to include small trib
            }

# Cats that were missed but should be associated with an awc lake record
#Probably need to use oid's for this one too
awcHuc12LakesQcLakesDict = {'Bristol_Bay_4115902':'325-30-10100-2249-3142-4163-0010', #Upnuk Lake
                            'Bristol_Bay_4105804':'325-30-10100-2031-3118-4212-5074-6011-0010', # LakeKulik
                            'Bristol_Bay_4100703':'325-30-10100-2249-3180-0010', #Lake Chauekuktuli
                            'Bristol_Bay_5032404':'326-00-10400-2058-0020', #Pungokepuk Lake
                            'Bristol_Bay_4101113':'325-30-10100-2249-0010', #Tikchik Lake
                            'Bristol_Bay_4093211':'325-30-10100-2202-3510-0010', #Twin Lakes
                            'Bristol_Bay_4085681':'325-30-10100-2249-3142-0010', #Nishlik Lake
                            'Bristol_Bay_4075956':'325-30-10100-2031-0010', #Aleknagik, Lake
                            'Bristol_Bay_4074196':'325-20-10030-0010', #Nunavaugaluk, Lake
                            'Bristol_Bay_4064277':'325-10-10010-0010', # Amanka Lake
                            'Bristol_Bay_3032395':'324-20-10140-0010', #Naknek Lake (Upper)
                            'Bristol_Bay_3024343':'324-20-10140-0010', #Naknek Lake (Outlet)
                            'Bristol_Bay_3023823':'324-20-10140-2245-3029-0020', # Coville Lake
                            'Bristol_Bay_2088766':'324-10-10150-0010', # Illiamna Lake
                            'Bristol_Bay_2088163':'324-10-10150-2207-0020', #Sixmile Lake
                            'Bristol_Bay_2087903':'324-10-10150-2207-0030', #Lake Clark
                            'Bristol_Bay_2086766':'324-10-10150-2010-0030', #Kukaklek Lake
                            'Bristol_Bay_2084672':'324-10-10150-2207-0040', #Little Lake Clark
                            'Bristol_Bay_2079856':'324-10-10150-2010-3115-4037-0020', #Spectacle Lake
                            'Bristol_Bay_2069087':'324-10-10150-2010-3070-0010', #Novnianuk Lake
                            'Bristol_Bay_2068597':'324-10-10150-2010-3070-0020', #Kulik Lake
                            'Bristol_Bay_1023034':'322-10-10080-0020', #Becharof Lake
                            }

print(f'QC Dictionaries set\n{"*"*100}')



QC Dictionaries set
****************************************************************************************************


In [77]:
# Relate awc code back to new catchment using one of the awc oids stored in the dictionary
# Copy this and add back to awcQC dictionary or modify update sp_ls code block below
for k,v in addHuc12QcDict.items():
    #print(k,v)
    awccode = awcOidDict[int(v[0])][1]
    print (f"'{k}':'{awccode}',")

'Cook_Inlet_75004200003619':'232-22-10130',
'Prince_William_Sound_78912':'222-10-12040',
'Prince_William_Sound_39255':'224-40-14904',
'Prince_William_Sound_60732':'223-30-13180',
'Copper_River_75003900054448':'228-70-10500',
'Copper_River_75003900084672':'212-20-10080-2100-3089',
'Cook_Inlet_75005300022592':'247-30-10090-2151',
'Cook_Inlet_75005400039351':'248-40-10105',
'Cook_Inlet_75000600015299':'247-41-10200-2053-3150',
'Cook_Inlet_75004200016346':'231-20-10337',
'Cook_Inlet_75004200012680':'233-30-10070',
'Cook_Inlet_75004200005429':'231-30-10080-2119',
'Cook_Inlet_75004200012771':'242-31-10120',
'Cook_Inlet_75004200006975':'226-50-16286',
'Cook_Inlet_75004300006970':'241-14-10645',
'Cook_Inlet_75004300002057':'241-14-10645-2131',
'Cook_Inlet_75004300007978':'241-14-10600',
'Cook_Inlet_75004300001135':'241-30-10500',
'Cook_Inlet_75004300006302':'241-20-10550',
'Cook_Inlet_75004300002183':'241-40-10300',
'Cook_Inlet_75004300005952':'241-14-10660',
'Cook_Inlet_75004400007224':'247-6

In [ ]:
# #'Cook_Inlet_75004200012620': ['Cook_Inlet_75004200001574', '232-21-10240', '190202021202', 2],
# reviewDict2={}
# with arcpy.da.SearchCursor(awcjoin,['cat_ID_con', 'HUC12'])as cur:
#     for row in cur:
#         if row[0] in wrongWtd:
#             newcat = wrongWtd[row[0]][0]
#             awccode = wrongWtd[row[0]][1]
#             #print(f'HUC {row[1]} catchment {row[0]} is incorrect and will be changed to {newcat} and awcEvent {awccode}')
#             append_value(reviewDict2,row[0],[newcat,awccode,row[1],2])
# for k, v in reviewDict2.items():
#     print(f"'{k}':{v},")


## Update using qc dictionaries after reviewing new watersheds
Use AWC_OID codes to update

In [79]:
# List to store catchments that were did not get attributed in spatial join but were identified in initial review as awc catchments
adQC = []
# List to store catchments without spawning habitat
nosphab = []
sp_ls_missvals = []
sa_sp_codes = ['CO_s','S_s','K_s','CH_s','P_s']
sa_no_sp_codes = ['CO_p','CO_r','S_p','S_r','K_p','K_r','CH_p','CH_r','P_r','P_p']
fields = ['cat_ID_con','AWC_CODE_qc','sp_ls','sa_sp_hab','AWC_OID','AWC_CODE','mult_awc','LSTAGE','SPECIES','sa_sp_count']

c=1
with arcpy.da.UpdateCursor(awcjoin,fields) as cur:
    for row in cur:
        if row[0] in awcHuc12QcDict:
            print(f'\n{("*"*100)}')
            print(f'{c}. {row[0]} found in awc qc dictionary')
            updCode = awcHuc12QcDict[row[0]]
            if updCode is not None:
                if row[4] is  not None:
                    awcOidList = []
                    awcSpLS = []
                    awcOIDvals = row[4].split(",")
                    print(f' oid vals  = {awcOIDvals} - row4 = {row[4]}')
                    awcCodevals = row[5].split(",")
                    print(f'code vals = {awcCodevals}')
                    snum = len(row[5].split(","))
                    for i in range(snum):
                        if awcCodevals[i] == updCode:
                            awcoid = int(awcOIDvals[i])
                            print(f'awcCode value of i = {awcCodevals[i]} and updCode = {updCode}\nAWC_OID = {awcoid}')
                            print(f'Appending {awcOidDict[int(awcOIDvals[i])][0]} from OID dictionary')
                            awcSpLS.append(awcOidDict[int(awcOIDvals[i])][0])
                    updSpLS = str(awcSpLS).strip("[]").replace("'","")
                    print (f'Awc codes {row[1]} will be updated to {updCode}')
                    print (f'Sp_ls codes {row[2]} will be updated to {updSpLS}')
                    if len(set(awcSpLS).intersection(sa_sp_codes)) >0:
                        print(f'{row[0]} has {len(set(awcSpLS).intersection(sa_sp_codes))} salmon spawning records')
                        saspcount = len(set(awcSpLS).intersection(sa_sp_codes))
                        row[3] = 1
                        row[9] = saspcount
                    elif len(set(awcSpLS).intersection(sa_no_sp_codes)) > 0  and len(set(awcSpLS).intersection(sa_sp_codes)) == 0:
                        print(f'{row[0]} has {len(set(awcSpLS).intersection(sa_no_sp_codes))} salmon species records')
                        row[3] = 2
                        row[9] = 0
                    else:
                        print(f'{row[0]} has no spawning habitat recorded')
                        nosphab.append(row[0])
                        row[3] = 0
                        row[9] = 0
                    row[1] = updCode
                    row[2] = updSpLS
                else:
                    sp_ls_missvals = []
                    print(f'{row[0]} missed during spatial join but identified during qc - must review again and add OBJECTIDs from awc to new keyval dictionary')
                    for val in addHuc12QcDict[row[0]]:
                        intval = int(val)
                        sp_ls_val = awcOidDict[intval][0]
                        sp_ls_missvals.append(sp_ls_val)
                    row[1] = updCode
                    # sp_ls
                    sp_ls_up = str(sp_ls_missvals).strip("[]").replace("\'","")
                    print(f'{sp_ls_up}')
                    row[2] = sp_ls_up
                    # sa_sp_hab
                    if len(set(sp_ls_missvals).intersection(sa_sp_codes)) >0:
                        print(f'{row[0]} has {len(set(sp_ls_missvals).intersection(sa_sp_codes))} salmon spawning records')
                        saspcount = len(set(sp_ls_missvals).intersection(sa_sp_codes))
                        row[3] = 1
                        row[9] = saspcount
                    elif len(set(sp_ls_missvals).intersection(sa_no_sp_codes)) > 0  and len(set(sp_ls_missvals).intersection(sa_sp_codes)) == 0:
                        print(f'{row[0]} has {len(set(sp_ls_missvals).intersection(sa_no_sp_codes))} salmon species records')
                        row[3] = 2
                        row[9] = 0
                    else:
                        print(f'{row[0]} has no spawning habitat recorded')
                        nosphab.append(row[0])
                        row[3] = 0
                        row[9] = 0
                    # AWC_OID
                    oids = str(addHuc12QcDict[row[0]]).strip("[]").replace("'","")
                    print(oids)
                    row[4] = oids
                    # AWC_CODE
                    print(len(addHuc12QcDict))
                    awccode = str(awcHuc12QcDict[row[0]].replace("'",""))

                    print(awccode)
                    row[5] = awccode
                    adQC.append(row[0])
            else:
                print(f'{("*"*100)}\n')
                print(f'{row[0]} will have code removed {updCode}')
                row[1]=None
                row[2]=None
                row[3]=None
                row[4]=None
                row[5]=None
                row[6]=None
                row[7]=None
                row[8]=None
                row[9]=None

            c+=1
        elif row[0] in awcHuc12LakesQcLakesDict:
            print(f'{("*"*100)}\n')
            print(f'{c}. {row[0]} found in awc lakes qc dictionary')
            print(f'{row[0]} found in awc qc dictionary')
            updCode = awcHuc12LakesQcLakesDict[row[0]]
            updSpLS = str(awcLakesDict[updCode]).strip("[]").replace("'","")
            print (f'Awc codes {row[1]} will be updated to {updCode}')
            print (f'Sp_ls codes {row[2]} will be updated to {updSpLS}')
            if len(set(awcLakesDict[updCode]).intersection(sa_sp_codes)) >0:
                print(f'{row[0]} has {len(set(awcLakesDict[updCode]).intersection(sa_sp_codes))} salmon spawning records')
                saspcount = len(set(awcLakesDict[updCode]).intersection(sa_sp_codes))
                row[3] = 1
                row[9] = saspcount
            elif len(set(awcLakesDict[updCode]).intersection(sa_no_sp_codes)) > 0  and len(set(awcLakesDict[updCode]).intersection(sa_sp_codes)) == 0:
                print(f'{row[0]} has {len(set(awcLakesDict[updCode]).intersection(sa_no_sp_codes))} salmon species records')
                row[3] = 2
                row[9] = 0
            else:
                print(f'{row[0]} has no spawning habitat recorded')
                row[3] = 0
                row[9] = 0
            row[1] = updCode
            row[2] = updSpLS
            c+=1
        else:
            pass
        cur.updateRow(row)
    del(row)
del(cur)


****************************************************************************************************
1. Cook_Inlet_75004200001493 found in awc qc dictionary
 oid vals  = ['20465', '20933', '21315', '22101'] - row4 = 20465,20933,21315,22101
code vals = ['233-20-10080', '233-20-10082', '233-20-10082', '233-20-10080']
awcCode value of i = 233-20-10080 and updCode = 233-20-10080
AWC_OID = 20465
Appending CH_s from OID dictionary
awcCode value of i = 233-20-10080 and updCode = 233-20-10080
AWC_OID = 22101
Appending P_s from OID dictionary
Awc codes 233-20-10080, 233-20-10082 will be updated to 233-20-10080
Sp_ls codes P_s, CH_s will be updated to CH_s, P_s
Cook_Inlet_75004200001493 has 2 salmon spawning records

****************************************************************************************************
2. Cook_Inlet_75004200003619 found in awc qc dictionary
Cook_Inlet_75004200003619 missed during spatial join but identified during qc - must review again and add OBJECTIDs from awc 

## Join fields to catchment centroid points using cat_ID_con
* Join Sp_Ls and sa_sp_hab to point fc of temperature sites

In [80]:
from time import strftime
print( "Start script: " + strftime("%Y-%m-%d %H:%M:%S"))

sitefields = [f.name for f in arcpy.ListFields(sites)]
awctextfields = ['sp_ls','AWC_CODE']
awcshortfields = ['sa_sp_hab','sa_sp_count']
sourceFieldsList = ['cat_ID_con','sp_ls','sa_sp_hab','sa_sp_count','AWC_CODE_qc']
updateFieldsList = ['cat_ID_con','sp_ls','sa_sp_hab','sa_sp_count','AWC_CODE']
# Use list comprehension to build a dictionary from a da SearchCursor
valueDict = {r[0]:(r[1:]) for r in arcpy.da.SearchCursor(awcjoin, sourceFieldsList)}

awcCatsDict = {}
for awctextfield in awctextfields:
    if awctextfield not in sitefields:
        print(f'adding field {awctextfield} to {sites}')
        arcpy.AddField_management(sites,awctextfield,'TEXT','','',500)
    else:
        print(f'{awctextfield} present')

for awcshortfield in awcshortfields:
    if awcshortfield not in sitefields:
        print(f'adding field {awcshortfield} to {sites}')
        arcpy.AddField_management(sites,awcshortfield,'SHORT')
    else:
        print(f'{awcshortfield} present')

with arcpy.da.UpdateCursor(sites,updateFieldsList) as cur:
    for row in cur:
        keyVal = row[0]
        if keyVal in valueDict:
            for n in range (1,len(sourceFieldsList)):
                row[n] = valueDict[keyVal][n-1]
            cur.updateRow(row)

del valueDict

print( "Finished script: " + strftime("%Y-%m-%d %H:%M:%S"))


Start script: 2022-04-15 14:57:30
sp_ls present
AWC_CODE present
sa_sp_hab present
sa_sp_count present
Finished script: 2022-04-15 14:57:30


## Export a csv copy


In [81]:
# Export watershed as table after join fields
akkssf_sp_ls_csv = arcpy.conversion.TableToTable(sites, outdir, 'AKSSF_awcHuc12s_SpLs.csv')
print('Export to csv complete')

Export to csv complete
